In [1]:
import numpy as np
from functools import partial
from pydrake import symbolic

from pydrake.all import (
    AddDefaultVisualization,
    Box,
    CoulombFriction,
    ProximityProperties,
    ContactResults,
    ContactModel,
    RigidTransform,
    ConnectContactResultsToDrakeVisualizer,
    GeometryInstance,
    DiscreteContactApproximation,
    PidController,
    RobotDiagramBuilder,
    Simulator,
    StartMeshcat,
    AddMultibodyPlantSceneGraph,
    AddUnitQuaternionConstraintOnPlant,
    AutoDiffXd,
    DiagramBuilder,
    ExtractGradient,
    ExtractValue,
    InitializeAutoDiff,
    JacobianWrtVariable,
    JointIndex,
    MathematicalProgram,
    MeshcatVisualizer,
    OrientationConstraint,
    Parser,
    PiecewisePolynomial,
    PositionConstraint,
    RotationMatrix,
    SnoptSolver,
    Solve, 
    eq,
    namedview,
)

from underactuated import ConfigureParser, running_as_notebook
from underactuated.multibody import MakePidStateProjectionMatrix    

# import os
# print(os.path.abspath(os.getcwd()))

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

In [ ]:
robot_builder = RobotDiagramBuilder(time_step=1e-5)

parser = robot_builder.parser()
ConfigureParser(parser)

parser.AddModels("groundblades.urdf")
parser.AddModels("spot4wd.dmd.yaml")

plant = robot_builder.plant()
plant.set_discrete_contact_approximation(DiscreteContactApproximation.kLagged)
plant.Finalize()

In [ ]:
# FORWARD SKATING WITH FOUR WHEELS
# should solve successfully with both gaits
# "skate" - refers to gait with back legs pushing
# "swizzle" - refers to gait with all legs on ground and pushing
# this specific simulation does not use a unique friction model and instead uses the dynamic nature of the wheeled "blades" to simulate the modified friction

def autoDiffArrayEqual(a, b):
    return np.array_equal(a, b) and np.array_equal(
        ExtractGradient(a), ExtractGradient(b)
    )

def gait_optimization(gait="walking_trot"):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    ConfigureParser(parser)
    (spot,) = parser.AddModels("spot4wd.dmd.yaml") #changed to spotblades
    parser.AddModels("groundblades.urdf")
    plant.Finalize()
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    #set_home(plant, plant_context)
    diagram.ForcedPublish(context)
    #print(context)
    q0 = plant.GetPositions(plant_context)
    body_frame = plant.GetFrameByName("body")
    PositionView = namedview(
        "Positions", plant.GetPositionNames(spot, always_add_suffix=False)
    )
    VelocityView = namedview(
        "Velocities",
        plant.GetVelocityNames(spot, always_add_suffix=False),
    )

    mu = 0.90 # rubber on rubber  #TODO: change friction coeff to 0.02 from 1
    total_mass = plant.CalcTotalMass(plant_context, [spot])
    gravity = plant.gravity_field().gravity_vector()

    nq = 12

    foot_frame = [
        plant.GetFrameByName("first_front_right_wheel"),
        plant.GetFrameByName("first_front_left_wheel"),
        plant.GetFrameByName("first_rear_right_wheel"),
        plant.GetFrameByName("first_rear_left_wheel"),
        plant.GetFrameByName("second_front_right_wheel"),
        plant.GetFrameByName("second_front_left_wheel"),
        plant.GetFrameByName("second_rear_right_wheel"),
        plant.GetFrameByName("second_rear_left_wheel"),
        plant.GetFrameByName("third_front_right_wheel"),
        plant.GetFrameByName("third_front_left_wheel"),
        plant.GetFrameByName("third_rear_right_wheel"),
        plant.GetFrameByName("third_rear_left_wheel"),
        plant.GetFrameByName("fourth_front_right_wheel"),
        plant.GetFrameByName("fourth_front_left_wheel"),
        plant.GetFrameByName("fourth_rear_right_wheel"),
        plant.GetFrameByName("fourth_rear_left_wheel")
    ]
    
    # second_foot_frame = [
    #     plant.GetFrameByName("second_front_right_wheel"),
    #     plant.GetFrameByName("second_front_left_wheel"),
    #     plant.GetFrameByName("second_rear_right_wheel"),
    #     plant.GetFrameByName("second_rear_left_wheel")
    # ]

    # third_foot_frame = [
    #     plant.GetFrameByName("third_front_right_wheel"),
    #     plant.GetFrameByName("third_front_left_wheel"),
    #     plant.GetFrameByName("third_rear_right_wheel"),
    #     plant.GetFrameByName("third_rear_left_wheel")
    # ]

    # fourth_foot_frame = [
    #     plant.GetFrameByName("fourth_front_right_wheel"),
    #     plant.GetFrameByName("fourth_front_left_wheel"),
    #     plant.GetFrameByName("fourth_rear_right_wheel"),
    #     plant.GetFrameByName("fourth_rear_left_wheel")
    # ]

    num_wheels = len(foot_frame)

    # setup gait
    is_laterally_symmetric = False
    check_self_collision = True
    if gait == "skate":
        N = 21
        in_stance = np.zeros((num_wheels, N))
        in_stance[0, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[1, 0:41] = 1
        in_stance[2, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[3, 0:18] = 1 #only right leg lifted for 26:N in stance
        in_stance[4, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[5, 0:41] = 1
        in_stance[6, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[7, 0:18] = 1 #only right leg lifted for 26:N in stance
        in_stance[8, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[9, 0:41] = 1
        in_stance[10, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[11, 0:18] = 1 #only right leg lifted for 26:N in stance
        in_stance[12, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[13, 0:41] = 1
        in_stance[14, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[15, 0:18] = 1 #only right leg lifted for 26:N in stance
        # # in_stance[2, 0:41] = 1 
        # # in_stance[3, 0:41] = 1
        # can't cross in front of torso
        speed = 0.7
        is_laterally_symmetric = True
        check_self_collision = True
        stride_length = 0.55
    elif gait == "swizzle":
        N = 21
        in_stance = np.ones((num_wheels, N))
        speed = 0.7
        is_laterally_symmetric = False
        check_self_collision = True
        stride_length = 0.55
    else:
        raise RuntimeError("Unknown gait.")

    T = stride_length / speed
    if is_laterally_symmetric:
        T = T / 2.0

    prog = MathematicalProgram()

    # Time steps
    h = prog.NewContinuousVariables(N - 1, "h")
    prog.AddBoundingBoxConstraint(0.5 * T / N, 2.0 * T / N, h) 
    prog.AddLinearConstraint(sum(h) >= 0.9 * T)
    prog.AddLinearConstraint(sum(h) <= 1.1 * T)

    # Create one context per time step (to maximize cache hits)
    context = [plant.CreateDefaultContext() for i in range(N)]
    #print(plant.GetPositionsAndVelocities(context[1]))

    # We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
    ad_plant = plant.ToAutoDiffXd()

    # Joint positions and velocities
    nq = plant.num_positions()
    nv = plant.num_velocities()
    q = prog.NewContinuousVariables(nq, N, "q")
    v = prog.NewContinuousVariables(nv, N, "v")
    q_view = PositionView(q)
    v_view = VelocityView(v)
    q0_view = PositionView(q0)
    # Joint costs
    q_cost = PositionView([1] * nq)
    v_cost = VelocityView([1] * nv)
    q_cost.body_x = 0
    q_cost.body_y = 0
    q_cost.body_qx = 0
    q_cost.body_qy = 0
    q_cost.body_qz = 0
    q_cost.body_qw = 0
    
    q_cost.front_left_hip_x = 0.7
    q_cost.front_right_hip_x = 0.7
    q_cost.rear_left_hip_x = 0.1
    q_cost.rear_right_hip_x = 0.1
    q_cost.rear_left_hip_y = 0.5
    q_cost.rear_right_hip_y = 0.5
    # q_cost.front_right_leg_to_platform = 0.5
    # q_cost.front_left_leg_to_platform = 0.5
    q_cost.rear_right_leg_to_platform = 0.5
    q_cost.rear_left_leg_to_platform = 0.5

    for n in range(N):
        # Joint limits
        prog.AddBoundingBoxConstraint(
            plant.GetPositionLowerLimits(),
            plant.GetPositionUpperLimits(),
            q[:, n],
        )
        # Joint velocity limits
        prog.AddBoundingBoxConstraint(
            plant.GetVelocityLowerLimits(),
            plant.GetVelocityUpperLimits(),
            v[:, n],
        )
        # Unit quaternions
        AddUnitQuaternionConstraintOnPlant(plant, q[:, n], prog)
        # Body orientation
        if n == 0:
            prog.AddConstraint(
                OrientationConstraint(
                    plant,
                    body_frame,
                    RotationMatrix(),
                    plant.world_frame(),
                    RotationMatrix(),
                    0.1,
                    context[n],
                ),
                q[:, n],
            )
        # Initial guess for all joint angles is the home position
        prog.SetInitialGuess(
            q[:, n], q0
        )  # Solvers get stuck if the quaternion is initialized with all zeros.

        # Running costs:
        prog.AddQuadraticErrorCost(np.diag(q_cost), q0, q[:, n])
        prog.AddQuadraticErrorCost(np.diag(v_cost), [0] * nv, v[:, n])

    # Make a new autodiff context for this constraint (to maximize cache hits)
    ad_velocity_dynamics_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def velocity_dynamics_constraint(vars, context_index):
        h, q, v, qn = np.split(vars, [1, 1 + nq, 1 + nq + nv])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                q,
                ad_plant.GetPositions(ad_velocity_dynamics_context[context_index]),
            ):
                ad_plant.SetPositions(ad_velocity_dynamics_context[context_index], q)
            v_from_qdot = ad_plant.MapQDotToVelocity(
                ad_velocity_dynamics_context[context_index], (qn - q) / h
            )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            v_from_qdot = plant.MapQDotToVelocity(context[context_index], (qn - q) / h)
        return v - v_from_qdot

    for n in range(N - 1):
        prog.AddConstraint(
            partial(velocity_dynamics_constraint, context_index=n),
            lb=[0] * nv,
            ub=[0] * nv,
            vars=np.concatenate(([h[n]], q[:, n], v[:, n], q[:, n + 1])),
        )

    # dyanmics of motion for the blades
    contact_force = [
        prog.NewContinuousVariables(3, N - 1, f"foot{foot}_contact_force")
        for foot in range(num_wheels)
    ]

    for n in range(N - 1):
        for foot in range(num_wheels):
            # Linear friction cone
            prog.AddLinearConstraint(
                contact_force[foot][0, n] <= 0.05 * mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                -contact_force[foot][0, n] <= 0.05 * mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                contact_force[foot][1, n] <=  mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                -contact_force[foot][1, n] <=  mu * contact_force[foot][2, n]
            )
            # normal force >=0, normal_force == 0 if not in_stance
            prog.AddBoundingBoxConstraint(
                0,
                in_stance[foot, n] * 4 * 9.81 * total_mass / 2,
                contact_force[foot][2, n],
            )
    

    # Center of mass variables and constraints
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N - 1, "comddot")
    # Initial CoM x,y position == 0
    prog.AddBoundingBoxConstraint(0, 0, com[:2, 0])
    # Initial CoM z vel == 0
    prog.AddBoundingBoxConstraint(0, 0, comdot[2, 0])
    # CoM height
    prog.AddBoundingBoxConstraint(0.125, np.inf, com[2, :])
    # CoM x velocity >= 0
    prog.AddBoundingBoxConstraint(0, np.inf, comdot[0, :])
    # CoM final x position
    if is_laterally_symmetric:
        prog.AddBoundingBoxConstraint(
            stride_length / 2.0, stride_length / 2.0, com[0, -1]
        )
    else:
        prog.AddBoundingBoxConstraint(stride_length, stride_length, com[0, -1])

    #adding cost to optimize for velocity
    for n in range(N-1):
        prog.AddQuadraticCost(-3*comdot[:,n].T @ comdot[:,n])
    
    prog.AddQuadraticCost(-com[:, n].T @ com[:, n])
    
    # prog.AddQuadraticCost(3*com[1:, n].T @ com[1:, n])

    # CoM dynamics
    for n in range(N - 1):
        # Note: The original matlab implementation used backwards Euler (here and throughout),
        # which is a little more consistent with the LCP contact models.
        prog.AddConstraint(eq(com[:, n + 1], com[:, n] + h[n] * comdot[:, n]))
        prog.AddConstraint(eq(comdot[:, n + 1], comdot[:, n] + h[n] * comddot[:, n]))
        prog.AddConstraint(
            eq(
                total_mass * comddot[:, n],
                sum(contact_force[i][:, n] for i in range(num_wheels)) + total_mass * gravity,
            )
        )
    
    # Angular momentum (about the center of mass)
    H = prog.NewContinuousVariables(3, N, "H")
    Hdot = prog.NewContinuousVariables(3, N - 1, "Hdot")
    prog.SetInitialGuess(H, np.zeros((3, N)))
    prog.SetInitialGuess(Hdot, np.zeros((3, N - 1)))

    # Hdot = sum_i cross(p_FootiW-com, contact_force_i)
    def angular_momentum_constraint(vars, context_index):
        q, com, Hdot, contact_force = np.split(vars, [nq, nq + 3, nq + 6])
        contact_force = contact_force.reshape(3, num_wheels, order="F")
        if isinstance(vars[0], AutoDiffXd):
            dq = ExtractGradient(q)
            q = ExtractValue(q)
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_wheels):
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    #[0, 0, -0.4325],
                    [0, 0, -0.03],
                    plant.world_frame(),
                )
                Jq_WF = plant.CalcJacobianTranslationalVelocity(
                    context[context_index],
                    JacobianWrtVariable.kQDot,
                    foot_frame[i],
                    #[0, 0, -0.4325],
                    [0, 0, -0.03],
                    plant.world_frame(),
                    plant.world_frame(),
                )
                ad_p_WF = InitializeAutoDiff(p_WF, Jq_WF @ dq)
                torque = torque + np.cross(
                    ad_p_WF.reshape(3) - com, contact_force[:, i]
                )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_wheels):
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    [0, 0, -0.03],
                    #[0, 0, -0.4325],
                    plant.world_frame(),
                )
                torque += np.cross(p_WF.reshape(3) - com, contact_force[:, i])
        return Hdot - torque

    # def TEST_CONSTRAINT(vars, context_index, foot_frame):
    #     #for n in range(N - 1):
    #     #q, com, Hdot, contact_force = np.split(vars, [nq, nq + 3, nq + 6])
    #     for foot in range(num_wheels):
    #         # dq = comdot[:, n]
    #         # i = foot
    #         # p_WF = plant.CalcPointsPositions(
    #         #         context[context_index],
    #         #         foot_frame[i],
    #         #         [0, 0, -0.03],
    #         #         plant.world_frame())
    #         # # Jq_WF = plant.CalcJacobianTranslationalVelocity(
    #         # #     context[context_index],
    #         # #     JacobianWrtVariable.kQDot,
    #         # #     foot_frame[i],
    #         # #     [0, 0, -0.03],
    #         # #     plant.world_frame(),
    #         # #     plant.world_frame())

    #         # # print(p_WF)
    #         # # print(Jq_WF)
    
    #         # axial_friction = [0.005, 0, 0]
    #         # lateral_friction = [0, 0.95, 0]

    #         # # Calculate normal force (e.g., based on skater's weight)
    #         # normal_force = in_stance[foot, context_index] * 4 * 9.81 * total_mass / 4

    #         # # Transform friction cone directions to the foot frame
    #         # wheel_angle = np.arctan(p_WF[1], p_WF[0])[0]
    #         # # print(wheel_angle)

    #         # rotation_matrix = np.array([[np.cos(wheel_angle), np.sin(wheel_angle), 0], 
    #         #                             [-np.sin(wheel_angle), np.cos(wheel_angle), 0], 
    #         #                             [0, 0, 1]])
    #         # friction_cone_x = rotation_matrix @ axial_friction
    #         # friction_cone_y = rotation_matrix @ lateral_friction
    #         # # friction_cone_x_foot = np.dot(RotationMatrix(), friction_cone_local_x)
    #         # # friction_cone_y_foot = np.dot(RotationMatrix(), friction_cone_local_y)
    #         # # print(friction_cone_x[0])

    #         # # Add friction cone constraints in foot frame
    #         # mu_x = np.abs(friction_cone_x[0] + friction_cone_y[0])
    #         # mu_y = np.abs(friction_cone_x[1] + friction_cone_y[1])

    #         foot_v = plant.CalcPointsSpatialVelocityInWorld(context[context_index], foot_frame[foot], contact_point)

    #             # if np.abs(foot_v[0]) > np.abs(foot_v[1]):
    #             #     friction = axial_friction
    #             # else:
    #             #     friction = lateral_friction
                
    #             normal_force = 4 * 9.81 * total_mass / 4

    #             foot_v_mag = np.linalg.norm(foot_v)
    #             if foot_v_mag < 0.01:
    #                 mu_x = axial_friction.static_friction
    #                 mu_y = lateral_friction.static_friction
    #             else:
    #                 mu_x = axial_friction.dynamic_friction
    #                 mu_y = lateral_friction.dynamic_friction

    #             # constraints.append(contact_force[foot][0, context_index] - mu * normal_force)
    #             fx = -mu_x * normal_force * foot_v[0] / (foot_v_mag + 1e-6)
    #             fy = -mu_y * normal_force * foot_v[1] / (foot_v_mag + 1e-6)
    #             constraints.append(contact_force[foot][0, context_index]-fx)
    #             constraints.append(contact_force[foot][1, context_index]-fy)

    #         prog.AddLinearConstraint(
    #             contact_force[foot][0, context_index] <= mu_x  * contact_force[foot][2, n]
    #         )
    #         prog.AddLinearConstraint(
    #             -contact_force[foot][0, context_index] <= mu_x * contact_force[foot][2, n]
    #         )
    #         prog.AddLinearConstraint(
    #             contact_force[foot][1, context_index] <=  mu_y * contact_force[foot][2, n]
    #         )
    #         prog.AddLinearConstraint(
    #             -contact_force[foot][1, context_index] <=  mu_y * contact_force[foot][2, n]
    #         )
    #         # normal force >=0, normal_force == 0 if not in_stance
    #         prog.AddBoundingBoxConstraint(
    #             0,
    #             in_stance[foot, context_index] * 4 * 9.81 * total_mass,
    #             contact_force[foot][2, context_index],
    #         )

    for n in range(N - 1):
        prog.AddConstraint(eq(H[:, n + 1], H[:, n] + h[n] * Hdot[:, n]))
        Fn = np.concatenate([contact_force[i][:, n] for i in range(num_wheels)])
        prog.AddConstraint(
            partial(angular_momentum_constraint, context_index=n),
            lb=np.zeros(3),
            ub=np.zeros(3),
            vars=np.concatenate((q[:, n], com[:, n], Hdot[:, n], Fn)),
        )
    
    # com == CenterOfMass(q), H = SpatialMomentumInWorldAboutPoint(q, v, com)
    # Make a new autodiff context for this constraint (to maximize cache hits)
    com_constraint_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def com_constraint(vars, context_index):
        qv, com, H = np.split(vars, [nq + nv, nq + nv + 3])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                qv,
                ad_plant.GetPositionsAndVelocities(com_constraint_context[context_index])
            ):
                ad_plant.SetPositionsAndVelocities(
                    com_constraint_context[context_index], qv
                )
            com_q = ad_plant.CalcCenterOfMassPositionInWorld(
                com_constraint_context[context_index], [spot]
            )
            H_qv = ad_plant.CalcSpatialMomentumInWorldAboutPoint(
                com_constraint_context[context_index], [spot], com
            ).rotational()
        else:
            if not np.array_equal(
                qv, plant.GetPositionsAndVelocities(context[context_index])
            ):
                plant.SetPositionsAndVelocities(context[context_index], qv)
            com_q = plant.CalcCenterOfMassPositionInWorld(
                context[context_index], [spot]
            )
            H_qv = plant.CalcSpatialMomentumInWorldAboutPoint(
                context[context_index], [spot], com
            ).rotational()
        return np.concatenate((com_q - com, H_qv - H))

    for n in range(N):
        prog.AddConstraint(
            partial(com_constraint, context_index=n),
            lb=np.zeros(6),
            ub=np.zeros(6),
            vars=np.concatenate((q[:, n], v[:, n], com[:, n], H[:, n])),
        )

    # TODO: Add collision constraints
    # Wheels cannot collide:

    # Kinematic constraints
    def fixed_position_constraint(vars, context_index, frame):
        q, qn = np.split(vars, [nq])
        if not np.array_equal(q, plant.GetPositions(context[context_index])):
            plant.SetPositions(context[context_index], q)
        if not np.array_equal(qn, plant.GetPositions(context[context_index + 1])):
            plant.SetPositions(context[context_index + 1], qn)
        p_WF = plant.CalcPointsPositions(
            context[context_index], frame, [0, 0, 0], plant.world_frame()
        )
        p_WF_n = plant.CalcPointsPositions(
            context[context_index + 1], frame, [0, 0, 0], plant.world_frame()
        )
        if isinstance(vars[0], AutoDiffXd):
            J_WF = plant.CalcJacobianTranslationalVelocity(
                context[context_index],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            J_WF_n = plant.CalcJacobianTranslationalVelocity(
                context[context_index + 1],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            return InitializeAutoDiff(
                p_WF_n - p_WF,
                J_WF_n @ ExtractGradient(qn) - J_WF @ ExtractGradient(q),
            )
        else:
            return p_WF_n - p_WF

    for i in range(num_wheels):
        for n in range(N):
            if in_stance[i, n]:
                # foot should be on the ground (world position z=0)
                if i % 2 == 0:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            [-np.inf, -np.inf, 0],
                            # [np.inf, np.inf, 0],
                            [np.inf, -0.01, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    # #SECOND WHEEL TRIAL RUN
                    # prog.AddConstraint( #added
                    #     PositionConstraint(
                    #         plant,
                    #         plant.world_frame(),
                    #         [-np.inf, -np.inf, 0],
                    #         # [np.inf, np.inf, 0],
                    #         [np.inf, np.inf, 0],
                    #         second_foot_frame[i],
                    #         #[0, 0, -0.4325],
                    #         [0, 0, -0.03],
                    #         context[n]),
                    #     q[:, n],
                    # )
                else:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            [-np.inf, 0.01, 0],
                            [np.inf, np.inf, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    # #SECOND WHEEL TRIAL RUN
                    # prog.AddConstraint( #added
                    #     PositionConstraint(
                    #         plant,
                    #         plant.world_frame(),
                    #         [-np.inf, 0.01, 0],
                    #         # [np.inf, np.inf, 0],
                    #         [np.inf, np.inf, 0],
                    #         second_foot_frame[i],
                    #         #[0, 0, -0.4325],
                    #         [0, 0, -0.03],
                    #         context[n]),
                    #     q[:, n],
                    # )
            else:
                min_clearance = 0.1
                prog.AddConstraint(
                    PositionConstraint(
                        plant,
                        plant.world_frame(),
                        [-np.inf, -np.inf, min_clearance], #TODO: edit to make feet on ground
                        [np.inf, np.inf, np.inf],
                        # [-np.inf, -np.inf, 0],
                        # [np.inf, np.inf, 0],
                        foot_frame[i],
                        #[0, 0, -0.4325],
                        [0, 0, -0.03],
                        context[n],),
                    q[:, n],
                )
                # #more second foot stuff
                # prog.AddConstraint( #added
                #     PositionConstraint(
                #         plant,
                #         plant.world_frame(),
                #         [-np.inf, -np.inf, min_clearance], #TODO: edit to make feet on ground
                #         [np.inf, np.inf, np.inf],
                #         # [-np.inf, -np.inf, 0],
                #         # [np.inf, np.inf, 0],
                #         second_foot_frame[i],
                #         #[0, 0, -0.4325],
                #         [0, 0, -0.03],
                #         context[n]),
                #     q[:, n],
                # )


  
    # Periodicity constraints
    if is_laterally_symmetric:
        # Joints
        def AddAntiSymmetricPair(a, b):
            prog.AddLinearEqualityConstraint(a[0] == -b[-1])
            prog.AddLinearEqualityConstraint(a[-1] == -b[0])

        def AddSymmetricPair(a, b):
            prog.AddLinearEqualityConstraint(a[0] == b[-1])
            prog.AddLinearEqualityConstraint(a[-1] == b[0])

        AddAntiSymmetricPair(q_view.front_left_hip_x, q_view.front_right_hip_x)
        AddSymmetricPair(q_view.front_left_hip_y, q_view.front_right_hip_y)
        AddSymmetricPair(q_view.front_left_knee, q_view.front_right_knee)
        AddAntiSymmetricPair(q_view.rear_left_hip_x, q_view.rear_right_hip_x)
        AddSymmetricPair(q_view.rear_left_hip_y, q_view.rear_right_hip_y)
        AddSymmetricPair(q_view.rear_left_knee, q_view.rear_right_knee)
        prog.AddLinearEqualityConstraint(q_view.body_y[0] == -q_view.body_y[-1])
        prog.AddLinearEqualityConstraint(q_view.body_z[0] == q_view.body_z[-1])
        # Body orientation must be in the xz plane:
        prog.AddBoundingBoxConstraint(0, 0, q_view.body_qx[[0, -1]])
        prog.AddBoundingBoxConstraint(0, 0, q_view.body_qz[[0, -1]])

        # Floating base velocity
        prog.AddLinearEqualityConstraint(v_view.body_vx[0] == v_view.body_vx[-1])
        prog.AddLinearEqualityConstraint(v_view.body_vy[0] == -v_view.body_vy[-1])
        prog.AddLinearEqualityConstraint(v_view.body_vz[0] == v_view.body_vz[-1])

        # # CoM velocity
        prog.AddLinearEqualityConstraint(comdot[0, 0] == comdot[0, -1])
        prog.AddLinearEqualityConstraint(comdot[1, 0] == -comdot[1, -1])
        prog.AddLinearEqualityConstraint(comdot[2, 0] == comdot[2, -1])

    else:
        # Everything except body_x is periodic
        q_selector = PositionView([True] * nq)
        q_selector.body_x = False
        prog.AddLinearConstraint(eq(q[q_selector, 0], q[q_selector, -1]))
        prog.AddLinearConstraint(eq(v[:, 0], v[:, -1]))

    # TODO: Set solver parameters (mostly to make the worst case solve times less bad)
    snopt = SnoptSolver().solver_id()
    prog.SetSolverOption(snopt, "Iterations Limits", 1e5 if running_as_notebook else 1)
    prog.SetSolverOption(
        snopt, "Major Iterations Limit", 200 if running_as_notebook else 1
    )
    prog.SetSolverOption(snopt, "Major Feasibility Tolerance", 5e-6)
    prog.SetSolverOption(snopt, "Major Optimality Tolerance", 1e-4)
    prog.SetSolverOption(snopt, "Superbasics limit", 2000)
    prog.SetSolverOption(snopt, "Linesearch tolerance", 0.9)
    # prog.SetSolverOption(snopt, 'Print file', 'snopt.out')

    result = Solve(prog)
    print(result.get_solver_id().name())
    print(result.is_success())  # We expect this to be false if iterations are limited.

    def HalfStrideToFullStride(a):
        b = PositionView(np.copy(a))

        b.body_y = -a.body_y
        # Mirror quaternion so that roll=-roll, yaw=-yaw
        b.body_qx = -a.body_qx
        b.body_qz = -a.body_qz

        b.front_left_hip_x = -a.front_right_hip_x
        b.front_right_hip_x = -a.front_left_hip_x
        b.rear_left_hip_x = -a.rear_right_hip_x
        b.rear_right_hip_x = -a.rear_left_hip_x

        b.front_left_hip_y = a.front_right_hip_y
        b.front_right_hip_y = a.front_left_hip_y
        b.rear_left_hip_y = a.rear_right_hip_y
        b.rear_right_hip_y = a.rear_left_hip_y

        b.front_left_knee = a.front_right_knee
        b.front_right_knee = a.front_left_knee
        b.rear_left_knee = a.rear_right_knee
        b.rear_right_knee = a.rear_left_knee

        return b

    # Animate trajectory
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    t_sol = np.cumsum(np.concatenate(([0], result.GetSolution(h))))
    q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
    visualizer.StartRecording()
    num_strides = 4
    t0 = t_sol[0]
    tf = t_sol[-1]
    T = tf * num_strides * (2.0 if is_laterally_symmetric else 1.0)
    for t in np.hstack((np.arange(t0, T, 1.0 / 32.0), T)):
        context.SetTime(t)
        stride = (t - t0) // (tf - t0)
        ts = (t - t0) % (tf - t0)
        qt = PositionView(q_sol.value(ts))
        if is_laterally_symmetric:
            if stride % 2 == 1:
                qt = HalfStrideToFullStride(qt)
                qt.body_x += stride_length / 2.0
            stride = stride // 2
        qt.body_x += stride * stride_length
        plant.SetPositions(plant_context, qt[:])
        diagram.ForcedPublish(context)

    visualizer.StopRecording()
    visualizer.PublishRecording()


gait_optimization("skate")

In [3]:
# FORWARD SKATING WITH TWO WHEELS
# should solve successfully with both gaits
# see further notes from code block above

def autoDiffArrayEqual(a, b):
    return np.array_equal(a, b) and np.array_equal(
        ExtractGradient(a), ExtractGradient(b)
    )

def gait_optimization(gait="walking_trot"):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    ConfigureParser(parser)
    (spot,) = parser.AddModels("spotblades.dmd.yaml") #changed to spotblades
    parser.AddModels("groundblades.urdf")
    plant.Finalize()
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    #set_home(plant, plant_context)
    diagram.ForcedPublish(context)
    #print(context)
    q0 = plant.GetPositions(plant_context)
    body_frame = plant.GetFrameByName("body")
    PositionView = namedview(
        "Positions", plant.GetPositionNames(spot, always_add_suffix=False)
    )
    VelocityView = namedview(
        "Velocities",
        plant.GetVelocityNames(spot, always_add_suffix=False),
    )

    mu = 0.90 # rubber on rubber  #TODO: change friction coeff to 0.02 from 1
    total_mass = plant.CalcTotalMass(plant_context, [spot])
    gravity = plant.gravity_field().gravity_vector()

    nq = 12

    foot_frame = [
        plant.GetFrameByName("first_front_right_wheel"),
        plant.GetFrameByName("first_front_left_wheel"),
        plant.GetFrameByName("first_rear_right_wheel"),
        plant.GetFrameByName("first_rear_left_wheel"),
        plant.GetFrameByName("second_front_right_wheel"),
        plant.GetFrameByName("second_front_left_wheel"),
        plant.GetFrameByName("second_rear_right_wheel"),
        plant.GetFrameByName("second_rear_left_wheel"),
    ]
    
    num_wheels = len(foot_frame)

    # setup gait
    is_laterally_symmetric = False
    check_self_collision = True
    if gait == "skate":
        N = 21
        in_stance = np.zeros((num_wheels, N))
        in_stance[0, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[1, 0:41] = 1
        in_stance[2, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[3, 0:18] = 1 #only right leg lifted for 26:N in stance
        in_stance[4, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[5, 0:41] = 1
        in_stance[6, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[7, 0:18] = 1 #only right leg lifted for 26:N in stance
        # can't cross in front of torso
        speed = 0.7
        is_laterally_symmetric = True
        check_self_collision = True
        stride_length = 0.55
    elif gait == "swizzle":
        N = 21
        in_stance = np.ones((num_wheels, N))
        speed = 0.7
        is_laterally_symmetric = False
        check_self_collision = True
        stride_length = 0.55
    else:
        raise RuntimeError("Unknown gait.")

    T = stride_length / speed
    if is_laterally_symmetric:
        T = T / 2.0

    prog = MathematicalProgram()

    # Time steps
    h = prog.NewContinuousVariables(N - 1, "h")
    prog.AddBoundingBoxConstraint(0.5 * T / N, 2.0 * T / N, h) 
    prog.AddLinearConstraint(sum(h) >= 0.9 * T)
    prog.AddLinearConstraint(sum(h) <= 1.1 * T)

    # Create one context per time step (to maximize cache hits)
    context = [plant.CreateDefaultContext() for i in range(N)]
    #print(plant.GetPositionsAndVelocities(context[1]))

    # We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
    ad_plant = plant.ToAutoDiffXd()

    # Joint positions and velocities
    nq = plant.num_positions()
    nv = plant.num_velocities()
    q = prog.NewContinuousVariables(nq, N, "q")
    v = prog.NewContinuousVariables(nv, N, "v")
    q_view = PositionView(q)
    v_view = VelocityView(v)
    q0_view = PositionView(q0)
    # Joint costs
    q_cost = PositionView([1] * nq)
    v_cost = VelocityView([1] * nv)
    q_cost.body_x = 0
    q_cost.body_y = 0
    q_cost.body_qx = 0
    q_cost.body_qy = 0
    q_cost.body_qz = 0
    q_cost.body_qw = 0
    
    q_cost.front_left_hip_x = 0.7
    q_cost.front_right_hip_x = 0.7
    q_cost.rear_left_hip_x = 0.1
    q_cost.rear_right_hip_x = 0.1
    q_cost.rear_left_hip_y = 0.5
    q_cost.rear_right_hip_y = 0.5
    # q_cost.front_right_leg_to_platform = 0.5
    # q_cost.front_left_leg_to_platform = 0.5
    q_cost.rear_right_leg_to_platform = 0.5
    q_cost.rear_left_leg_to_platform = 0.5

    for n in range(N):
        # Joint limits
        prog.AddBoundingBoxConstraint(
            plant.GetPositionLowerLimits(),
            plant.GetPositionUpperLimits(),
            q[:, n],
        )
        # Joint velocity limits
        prog.AddBoundingBoxConstraint(
            plant.GetVelocityLowerLimits(),
            plant.GetVelocityUpperLimits(),
            v[:, n],
        )
        # Unit quaternions
        AddUnitQuaternionConstraintOnPlant(plant, q[:, n], prog)
        # Body orientation
        if n == 0:
            prog.AddConstraint(
                OrientationConstraint(
                    plant,
                    body_frame,
                    RotationMatrix(),
                    plant.world_frame(),
                    RotationMatrix(),
                    0.1,
                    context[n],
                ),
                q[:, n],
            )
        # Initial guess for all joint angles is the home position
        prog.SetInitialGuess(
            q[:, n], q0
        )  # Solvers get stuck if the quaternion is initialized with all zeros.

        # Running costs:
        prog.AddQuadraticErrorCost(np.diag(q_cost), q0, q[:, n])
        prog.AddQuadraticErrorCost(np.diag(v_cost), [0] * nv, v[:, n])

    # Make a new autodiff context for this constraint (to maximize cache hits)
    ad_velocity_dynamics_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def velocity_dynamics_constraint(vars, context_index):
        h, q, v, qn = np.split(vars, [1, 1 + nq, 1 + nq + nv])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                q,
                ad_plant.GetPositions(ad_velocity_dynamics_context[context_index]),
            ):
                ad_plant.SetPositions(ad_velocity_dynamics_context[context_index], q)
            v_from_qdot = ad_plant.MapQDotToVelocity(
                ad_velocity_dynamics_context[context_index], (qn - q) / h
            )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            v_from_qdot = plant.MapQDotToVelocity(context[context_index], (qn - q) / h)
        return v - v_from_qdot

    for n in range(N - 1):
        prog.AddConstraint(
            partial(velocity_dynamics_constraint, context_index=n),
            lb=[0] * nv,
            ub=[0] * nv,
            vars=np.concatenate(([h[n]], q[:, n], v[:, n], q[:, n + 1])),
        )

    # dyanmics of motion for the blades
    contact_force = [
        prog.NewContinuousVariables(3, N - 1, f"foot{foot}_contact_force")
        for foot in range(num_wheels)
    ]

    for n in range(N - 1):
        for foot in range(num_wheels):
            # Linear friction cone
            prog.AddLinearConstraint(
                contact_force[foot][0, n] <= 0.05 * mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                -contact_force[foot][0, n] <= 0.05 * mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                contact_force[foot][1, n] <=  mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                -contact_force[foot][1, n] <=  mu * contact_force[foot][2, n]
            )
            # normal force >=0, normal_force == 0 if not in_stance
            prog.AddBoundingBoxConstraint(
                0,
                in_stance[foot, n] * 4 * 9.81 * total_mass / 2,
                contact_force[foot][2, n],
            )
    

    # Center of mass variables and constraints
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N - 1, "comddot")
    # Initial CoM x,y position == 0
    prog.AddBoundingBoxConstraint(0, 0, com[:2, 0])
    # Initial CoM z vel == 0
    prog.AddBoundingBoxConstraint(0, 0, comdot[2, 0])
    # CoM height
    prog.AddBoundingBoxConstraint(0.125, np.inf, com[2, :])
    # CoM x velocity >= 0
    prog.AddBoundingBoxConstraint(0, np.inf, comdot[0, :])
    # CoM final x position
    if is_laterally_symmetric:
        prog.AddBoundingBoxConstraint(
            stride_length / 2.0, stride_length / 2.0, com[0, -1]
        )
    else:
        prog.AddBoundingBoxConstraint(stride_length, stride_length, com[0, -1])

    #adding cost to optimize for velocity
    for n in range(N-1):
        prog.AddQuadraticCost(-3*comdot[:,n].T @ comdot[:,n])
    
    prog.AddQuadraticCost(-com[:, n].T @ com[:, n])
    
    # prog.AddQuadraticCost(3*com[1:, n].T @ com[1:, n])

    # CoM dynamics
    for n in range(N - 1):
        # Note: The original matlab implementation used backwards Euler (here and throughout),
        # which is a little more consistent with the LCP contact models.
        prog.AddConstraint(eq(com[:, n + 1], com[:, n] + h[n] * comdot[:, n]))
        prog.AddConstraint(eq(comdot[:, n + 1], comdot[:, n] + h[n] * comddot[:, n]))
        prog.AddConstraint(
            eq(
                total_mass * comddot[:, n],
                sum(contact_force[i][:, n] for i in range(num_wheels)) + total_mass * gravity,
            )
        )
    
    # Angular momentum (about the center of mass)
    H = prog.NewContinuousVariables(3, N, "H")
    Hdot = prog.NewContinuousVariables(3, N - 1, "Hdot")
    prog.SetInitialGuess(H, np.zeros((3, N)))
    prog.SetInitialGuess(Hdot, np.zeros((3, N - 1)))

    # Hdot = sum_i cross(p_FootiW-com, contact_force_i)
    def angular_momentum_constraint(vars, context_index):
        q, com, Hdot, contact_force = np.split(vars, [nq, nq + 3, nq + 6])
        contact_force = contact_force.reshape(3, num_wheels, order="F")
        if isinstance(vars[0], AutoDiffXd):
            dq = ExtractGradient(q)
            q = ExtractValue(q)
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_wheels):
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    #[0, 0, -0.4325],
                    [0, 0, -0.03],
                    plant.world_frame(),
                )
                Jq_WF = plant.CalcJacobianTranslationalVelocity(
                    context[context_index],
                    JacobianWrtVariable.kQDot,
                    foot_frame[i],
                    #[0, 0, -0.4325],
                    [0, 0, -0.03],
                    plant.world_frame(),
                    plant.world_frame(),
                )
                ad_p_WF = InitializeAutoDiff(p_WF, Jq_WF @ dq)
                torque = torque + np.cross(
                    ad_p_WF.reshape(3) - com, contact_force[:, i]
                )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_wheels):
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    [0, 0, -0.03],
                    #[0, 0, -0.4325],
                    plant.world_frame(),
                )
                torque += np.cross(p_WF.reshape(3) - com, contact_force[:, i])
        return Hdot - torque

    # def TEST_CONSTRAINT(vars, context_index, foot_frame):
    #     #for n in range(N - 1):
    #     #q, com, Hdot, contact_force = np.split(vars, [nq, nq + 3, nq + 6])
    #     for foot in range(num_wheels):
    #         # dq = comdot[:, n]
    #         # i = foot
    #         # p_WF = plant.CalcPointsPositions(
    #         #         context[context_index],
    #         #         foot_frame[i],
    #         #         [0, 0, -0.03],
    #         #         plant.world_frame())
    #         # # Jq_WF = plant.CalcJacobianTranslationalVelocity(
    #         # #     context[context_index],
    #         # #     JacobianWrtVariable.kQDot,
    #         # #     foot_frame[i],
    #         # #     [0, 0, -0.03],
    #         # #     plant.world_frame(),
    #         # #     plant.world_frame())

    #         # # print(p_WF)
    #         # # print(Jq_WF)
    
    #         # axial_friction = [0.005, 0, 0]
    #         # lateral_friction = [0, 0.95, 0]

    #         # # Calculate normal force (e.g., based on skater's weight)
    #         # normal_force = in_stance[foot, context_index] * 4 * 9.81 * total_mass / 4

    #         # # Transform friction cone directions to the foot frame
    #         # wheel_angle = np.arctan(p_WF[1], p_WF[0])[0]
    #         # # print(wheel_angle)

    #         # rotation_matrix = np.array([[np.cos(wheel_angle), np.sin(wheel_angle), 0], 
    #         #                             [-np.sin(wheel_angle), np.cos(wheel_angle), 0], 
    #         #                             [0, 0, 1]])
    #         # friction_cone_x = rotation_matrix @ axial_friction
    #         # friction_cone_y = rotation_matrix @ lateral_friction
    #         # # friction_cone_x_foot = np.dot(RotationMatrix(), friction_cone_local_x)
    #         # # friction_cone_y_foot = np.dot(RotationMatrix(), friction_cone_local_y)
    #         # # print(friction_cone_x[0])

    #         # # Add friction cone constraints in foot frame
    #         # mu_x = np.abs(friction_cone_x[0] + friction_cone_y[0])
    #         # mu_y = np.abs(friction_cone_x[1] + friction_cone_y[1])

    #         foot_v = plant.CalcPointsSpatialVelocityInWorld(context[context_index], foot_frame[foot], contact_point)

    #             # if np.abs(foot_v[0]) > np.abs(foot_v[1]):
    #             #     friction = axial_friction
    #             # else:
    #             #     friction = lateral_friction
                
    #             normal_force = 4 * 9.81 * total_mass / 4

    #             foot_v_mag = np.linalg.norm(foot_v)
    #             if foot_v_mag < 0.01:
    #                 mu_x = axial_friction.static_friction
    #                 mu_y = lateral_friction.static_friction
    #             else:
    #                 mu_x = axial_friction.dynamic_friction
    #                 mu_y = lateral_friction.dynamic_friction

    #             # constraints.append(contact_force[foot][0, context_index] - mu * normal_force)
    #             fx = -mu_x * normal_force * foot_v[0] / (foot_v_mag + 1e-6)
    #             fy = -mu_y * normal_force * foot_v[1] / (foot_v_mag + 1e-6)
    #             constraints.append(contact_force[foot][0, context_index]-fx)
    #             constraints.append(contact_force[foot][1, context_index]-fy)

    #         prog.AddLinearConstraint(
    #             contact_force[foot][0, context_index] <= mu_x  * contact_force[foot][2, n]
    #         )
    #         prog.AddLinearConstraint(
    #             -contact_force[foot][0, context_index] <= mu_x * contact_force[foot][2, n]
    #         )
    #         prog.AddLinearConstraint(
    #             contact_force[foot][1, context_index] <=  mu_y * contact_force[foot][2, n]
    #         )
    #         prog.AddLinearConstraint(
    #             -contact_force[foot][1, context_index] <=  mu_y * contact_force[foot][2, n]
    #         )
    #         # normal force >=0, normal_force == 0 if not in_stance
    #         prog.AddBoundingBoxConstraint(
    #             0,
    #             in_stance[foot, context_index] * 4 * 9.81 * total_mass,
    #             contact_force[foot][2, context_index],
    #         )

    for n in range(N - 1):
        prog.AddConstraint(eq(H[:, n + 1], H[:, n] + h[n] * Hdot[:, n]))
        Fn = np.concatenate([contact_force[i][:, n] for i in range(num_wheels)])
        prog.AddConstraint(
            partial(angular_momentum_constraint, context_index=n),
            lb=np.zeros(3),
            ub=np.zeros(3),
            vars=np.concatenate((q[:, n], com[:, n], Hdot[:, n], Fn)),
        )
    
    # com == CenterOfMass(q), H = SpatialMomentumInWorldAboutPoint(q, v, com)
    # Make a new autodiff context for this constraint (to maximize cache hits)
    com_constraint_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def com_constraint(vars, context_index):
        qv, com, H = np.split(vars, [nq + nv, nq + nv + 3])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                qv,
                ad_plant.GetPositionsAndVelocities(com_constraint_context[context_index])
            ):
                ad_plant.SetPositionsAndVelocities(
                    com_constraint_context[context_index], qv
                )
            com_q = ad_plant.CalcCenterOfMassPositionInWorld(
                com_constraint_context[context_index], [spot]
            )
            H_qv = ad_plant.CalcSpatialMomentumInWorldAboutPoint(
                com_constraint_context[context_index], [spot], com
            ).rotational()
        else:
            if not np.array_equal(
                qv, plant.GetPositionsAndVelocities(context[context_index])
            ):
                plant.SetPositionsAndVelocities(context[context_index], qv)
            com_q = plant.CalcCenterOfMassPositionInWorld(
                context[context_index], [spot]
            )
            H_qv = plant.CalcSpatialMomentumInWorldAboutPoint(
                context[context_index], [spot], com
            ).rotational()
        return np.concatenate((com_q - com, H_qv - H))

    for n in range(N):
        prog.AddConstraint(
            partial(com_constraint, context_index=n),
            lb=np.zeros(6),
            ub=np.zeros(6),
            vars=np.concatenate((q[:, n], v[:, n], com[:, n], H[:, n])),
        )

    # TODO: Add collision constraints
    # Wheels cannot collide:

    # Kinematic constraints
    def fixed_position_constraint(vars, context_index, frame):
        q, qn = np.split(vars, [nq])
        if not np.array_equal(q, plant.GetPositions(context[context_index])):
            plant.SetPositions(context[context_index], q)
        if not np.array_equal(qn, plant.GetPositions(context[context_index + 1])):
            plant.SetPositions(context[context_index + 1], qn)
        p_WF = plant.CalcPointsPositions(
            context[context_index], frame, [0, 0, 0], plant.world_frame()
        )
        p_WF_n = plant.CalcPointsPositions(
            context[context_index + 1], frame, [0, 0, 0], plant.world_frame()
        )
        if isinstance(vars[0], AutoDiffXd):
            J_WF = plant.CalcJacobianTranslationalVelocity(
                context[context_index],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            J_WF_n = plant.CalcJacobianTranslationalVelocity(
                context[context_index + 1],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            return InitializeAutoDiff(
                p_WF_n - p_WF,
                J_WF_n @ ExtractGradient(qn) - J_WF @ ExtractGradient(q),
            )
        else:
            return p_WF_n - p_WF

    for i in range(num_wheels):
        for n in range(N):
            if in_stance[i, n]:
                # foot should be on the ground (world position z=0)
                if i % 2 == 0:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            [-np.inf, -np.inf, 0],
                            # [np.inf, np.inf, 0],
                            [np.inf, -0.01, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    # #SECOND WHEEL TRIAL RUN
                    # prog.AddConstraint( #added
                    #     PositionConstraint(
                    #         plant,
                    #         plant.world_frame(),
                    #         [-np.inf, -np.inf, 0],
                    #         # [np.inf, np.inf, 0],
                    #         [np.inf, np.inf, 0],
                    #         second_foot_frame[i],
                    #         #[0, 0, -0.4325],
                    #         [0, 0, -0.03],
                    #         context[n]),
                    #     q[:, n],
                    # )
                else:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            [-np.inf, 0.01, 0],
                            [np.inf, np.inf, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    # #SECOND WHEEL TRIAL RUN
                    # prog.AddConstraint( #added
                    #     PositionConstraint(
                    #         plant,
                    #         plant.world_frame(),
                    #         [-np.inf, 0.01, 0],
                    #         # [np.inf, np.inf, 0],
                    #         [np.inf, np.inf, 0],
                    #         second_foot_frame[i],
                    #         #[0, 0, -0.4325],
                    #         [0, 0, -0.03],
                    #         context[n]),
                    #     q[:, n],
                    # )
            else:
                min_clearance = 0.1
                prog.AddConstraint(
                    PositionConstraint(
                        plant,
                        plant.world_frame(),
                        [-np.inf, -np.inf, min_clearance], #TODO: edit to make feet on ground
                        [np.inf, np.inf, np.inf],
                        # [-np.inf, -np.inf, 0],
                        # [np.inf, np.inf, 0],
                        foot_frame[i],
                        #[0, 0, -0.4325],
                        [0, 0, -0.03],
                        context[n],),
                    q[:, n],
                )
                # #more second foot stuff
                # prog.AddConstraint( #added
                #     PositionConstraint(
                #         plant,
                #         plant.world_frame(),
                #         [-np.inf, -np.inf, min_clearance], #TODO: edit to make feet on ground
                #         [np.inf, np.inf, np.inf],
                #         # [-np.inf, -np.inf, 0],
                #         # [np.inf, np.inf, 0],
                #         second_foot_frame[i],
                #         #[0, 0, -0.4325],
                #         [0, 0, -0.03],
                #         context[n]),
                #     q[:, n],
                # )


  
    # Periodicity constraints
    if is_laterally_symmetric:
        # Joints
        def AddAntiSymmetricPair(a, b):
            prog.AddLinearEqualityConstraint(a[0] == -b[-1])
            prog.AddLinearEqualityConstraint(a[-1] == -b[0])

        def AddSymmetricPair(a, b):
            prog.AddLinearEqualityConstraint(a[0] == b[-1])
            prog.AddLinearEqualityConstraint(a[-1] == b[0])

        AddAntiSymmetricPair(q_view.front_left_hip_x, q_view.front_right_hip_x)
        AddSymmetricPair(q_view.front_left_hip_y, q_view.front_right_hip_y)
        AddSymmetricPair(q_view.front_left_knee, q_view.front_right_knee)
        AddAntiSymmetricPair(q_view.rear_left_hip_x, q_view.rear_right_hip_x)
        AddSymmetricPair(q_view.rear_left_hip_y, q_view.rear_right_hip_y)
        AddSymmetricPair(q_view.rear_left_knee, q_view.rear_right_knee)
        prog.AddLinearEqualityConstraint(q_view.body_y[0] == -q_view.body_y[-1])
        prog.AddLinearEqualityConstraint(q_view.body_z[0] == q_view.body_z[-1])
        # Body orientation must be in the xz plane:
        prog.AddBoundingBoxConstraint(0, 0, q_view.body_qx[[0, -1]])
        prog.AddBoundingBoxConstraint(0, 0, q_view.body_qz[[0, -1]])

        # Floating base velocity
        prog.AddLinearEqualityConstraint(v_view.body_vx[0] == v_view.body_vx[-1])
        prog.AddLinearEqualityConstraint(v_view.body_vy[0] == -v_view.body_vy[-1])
        prog.AddLinearEqualityConstraint(v_view.body_vz[0] == v_view.body_vz[-1])

        # # CoM velocity
        prog.AddLinearEqualityConstraint(comdot[0, 0] == comdot[0, -1])
        prog.AddLinearEqualityConstraint(comdot[1, 0] == -comdot[1, -1])
        prog.AddLinearEqualityConstraint(comdot[2, 0] == comdot[2, -1])

    else:
        # Everything except body_x is periodic
        q_selector = PositionView([True] * nq)
        q_selector.body_x = False
        prog.AddLinearConstraint(eq(q[q_selector, 0], q[q_selector, -1]))
        prog.AddLinearConstraint(eq(v[:, 0], v[:, -1]))

    # TODO: Set solver parameters (mostly to make the worst case solve times less bad)
    snopt = SnoptSolver().solver_id()
    prog.SetSolverOption(snopt, "Iterations Limits", 1e5 if running_as_notebook else 1)
    prog.SetSolverOption(
        snopt, "Major Iterations Limit", 200 if running_as_notebook else 1
    )
    prog.SetSolverOption(snopt, "Major Feasibility Tolerance", 5e-6)
    prog.SetSolverOption(snopt, "Major Optimality Tolerance", 1e-4)
    prog.SetSolverOption(snopt, "Superbasics limit", 2000)
    prog.SetSolverOption(snopt, "Linesearch tolerance", 0.9)
    # prog.SetSolverOption(snopt, 'Print file', 'snopt.out')

    result = Solve(prog)
    print(result.get_solver_id().name())
    print(result.is_success())  # We expect this to be false if iterations are limited.

    def HalfStrideToFullStride(a):
        b = PositionView(np.copy(a))

        b.body_y = -a.body_y
        # Mirror quaternion so that roll=-roll, yaw=-yaw
        b.body_qx = -a.body_qx
        b.body_qz = -a.body_qz

        b.front_left_hip_x = -a.front_right_hip_x
        b.front_right_hip_x = -a.front_left_hip_x
        b.rear_left_hip_x = -a.rear_right_hip_x
        b.rear_right_hip_x = -a.rear_left_hip_x

        b.front_left_hip_y = a.front_right_hip_y
        b.front_right_hip_y = a.front_left_hip_y
        b.rear_left_hip_y = a.rear_right_hip_y
        b.rear_right_hip_y = a.rear_left_hip_y

        b.front_left_knee = a.front_right_knee
        b.front_right_knee = a.front_left_knee
        b.rear_left_knee = a.rear_right_knee
        b.rear_right_knee = a.rear_left_knee

        return b

    # Animate trajectory
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    t_sol = np.cumsum(np.concatenate(([0], result.GetSolution(h))))
    q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
    visualizer.StartRecording()
    num_strides = 4
    t0 = t_sol[0]
    tf = t_sol[-1]
    T = tf * num_strides * (2.0 if is_laterally_symmetric else 1.0)
    for t in np.hstack((np.arange(t0, T, 1.0 / 32.0), T)):
        context.SetTime(t)
        stride = (t - t0) // (tf - t0)
        ts = (t - t0) % (tf - t0)
        qt = PositionView(q_sol.value(ts))
        if is_laterally_symmetric:
            if stride % 2 == 1:
                qt = HalfStrideToFullStride(qt)
                qt.body_x += stride_length / 2.0
            stride = stride // 2
        qt.body_x += stride * stride_length
        plant.SetPositions(plant_context, qt[:])
        diagram.ForcedPublish(context)

    visualizer.StopRecording()
    visualizer.PublishRecording()


gait_optimization("skate")

In [ ]:
# Code block for experimentation

def autoDiffArrayEqual(a, b):
    return np.array_equal(a, b) and np.array_equal(
        ExtractGradient(a), ExtractGradient(b)
    )

def gait_optimization(gait="walking_trot", initital_pos = None):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    ConfigureParser(parser)
    (spot,) = parser.AddModels("spotblades.dmd.yaml") #changed to spotblades
    plant.Finalize()
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    #set_home(plant, plant_context)
    diagram.ForcedPublish(context)

    q0 = plant.GetPositions(plant_context)
    body_frame = plant.GetFrameByName("body")

    PositionView = namedview(
        "Positions", plant.GetPositionNames(spot, always_add_suffix=False)
    )
    VelocityView = namedview(
        "Velocities",
        plant.GetVelocityNames(spot, always_add_suffix=False),
    )

    mu = 0.95 # rubber on rubber  #TODO: change friction coeff to 0.02 from 1
    total_mass = plant.CalcTotalMass(plant_context, [spot])
    gravity = plant.gravity_field().gravity_vector()

    foot_frame = [
        plant.GetFrameByName("first_front_right_wheel"),
        plant.GetFrameByName("first_front_left_wheel"),
        plant.GetFrameByName("first_rear_right_wheel"),
        plant.GetFrameByName("first_rear_left_wheel"),
        plant.GetFrameByName("second_front_right_wheel"),
        plant.GetFrameByName("second_front_left_wheel"),
        plant.GetFrameByName("second_rear_right_wheel"),
        plant.GetFrameByName("second_rear_left_wheel")
    ]

    num_feet = len(foot_frame)

    foot_indices = [
        plant.GetJointByName("front_right_leg_to_platform").index(),
        plant.GetJointByName("front_left_leg_to_platform").index(),
        plant.GetJointByName("rear_right_leg_to_platform").index(),
        plant.GetJointByName("rear_left_leg_to_platform").index()
    ]

    platform_frame = [
        plant.GetFrameByName("front_right_wheel_platform"),
        plant.GetFrameByName("front_left_wheel_platform"),
        plant.GetFrameByName("rear_right_wheel_platform"),
        plant.GetFrameByName("rear_left_wheel_platform")
    ]

    leg_frame = [
        plant.GetFrameByName("front_right_lower_leg"),
        plant.GetFrameByName("front_left_lower_leg"),
        plant.GetFrameByName("rear_right_lower_leg"),
        plant.GetFrameByName("rear_left_lower_leg")
    ]

    ground_frame = plant.GetFrameByName("ground")
    ground_index = ground_frame.index()

    # setup gait
    is_laterally_symmetric = False
    check_self_collision = True
    if gait == "skate":
        N = 21
        in_stance = np.zeros((4, N))
        in_stance[0, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[1, 0:41] = 1
        in_stance[2, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[3, 0:18] = 1 #only right leg lifted for 26:N in stance
        # in_stance[2, 0:41] = 1 
        # in_stance[3, 0:41] = 1
        # can't cross in front of torso
        speed = 0.7
        is_laterally_symmetric = True
        check_self_collision = True
        stride_length = 0.55
    elif gait == "swizzle":
        N = 21
        in_stance = np.zeros((4, N))
        in_stance[0, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[1, 0:41] = 1
        in_stance[2, 0:41] = 1 
        in_stance[3, 0:41] = 1
        # can't cross in front of torso
        speed = 0.7
        is_laterally_symmetric = False
        is_wheel_symmetric = False
        check_self_collision = True
        stride_length = 0.55
    elif gait == "stop":
        N = 21
        in_stance = np.ones((4, N))
        speed = 0.02
        is_laterally_symmetric = True
        check_self_collision = True
        strid3_length = 0.3
    else:
        raise RuntimeError("Unknown gait.")

    T = stride_length / speed
    if is_laterally_symmetric:
        T = T / 2.0

    prog = MathematicalProgram()

    # Time steps
    h = prog.NewContinuousVariables(N - 1, "h")
    prog.AddBoundingBoxConstraint(0.5 * T / N, 2.0 * T / N, h) 
    prog.AddLinearConstraint(sum(h) >= 0.9 * T)
    prog.AddLinearConstraint(sum(h) <= 1.1 * T)

    # Create one context per time step (to maximize cache hits)
    context = [plant.CreateDefaultContext() for i in range(N)]
    # We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
    ad_plant = plant.ToAutoDiffXd()

    # Joint positions and velocities
    nq = plant.num_positions()
    nv = plant.num_velocities()
    q = prog.NewContinuousVariables(nq, N, "q")
    v = prog.NewContinuousVariables(nv, N, "v")
    q_view = PositionView(q)
    v_view = VelocityView(v)
    q0_view = PositionView(q0)
    # Joint costs
    # print(q_cost)
    # print(v_view)
    q_cost = PositionView([1] * nq)
    print(q_cost)
    v_cost = VelocityView([1] * nv)
    q_cost.body_x = 0
    q_cost.body_y = 0
    q_cost.body_qx = 0
    q_cost.body_qy = 0
    q_cost.body_qz = 0
    q_cost.body_qw = 0
    
    q_cost.front_left_hip_x = 0.2
    q_cost.front_right_hip_x = 0.2
    q_cost.rear_left_hip_x = 0.2
    q_cost.rear_right_hip_x = 0.2
    v_cost.body_vx = 0
    v_cost.body_wx = 0
    v_cost.body_wy = 0
    v_cost.body_wz = 0

    q_cost.front_right_leg_to_wheel = 0.1
    q_cost.second_front_right_leg_to_wheel = 0.1
    q_cost.front_left_leg_to_wheel = 0.1
    q_cost.second_front_left_leg_to_wheel = 0.1
    # q_cost.rear_right_leg_to_wheel = 0.1
    # q_cost.second_rear_right_leg_to_wheel = 0.1
    # q_cost.rear_right_leg_to_wheel = 0.1
    # q_cost.second_rear_left_leg_to_wheel = 0.1
    # q_cost.front_right_leg_to_platform = 0.1
    # q_cost.front_left_leg_to_platform = 0.1
    # q_cost.rear_right_leg_to_platform = 0.1
    # q_cost.rear_left_leg_to_platform = 0.1

    # v_cost.front_right_leg_to_wheel = 0.1
    # v_cost.second_front_right_leg_to_wheel = 0.1
    # v_cost.front_left_leg_to_wheel = 0.1
    # v_cost.second_front_left_leg_to_wheel = 0.1
    # v_cost.rear_right_leg_to_wheel = 0.1
    # v_cost.second_rear_right_leg_to_wheel = 0.1
    # v_cost.rear_right_leg_to_wheel = 0.1
    # v_cost.second_rear_left_leg_to_wheel = 0.1
    # v_cost.front_right_leg_to_platform = 0.1
    # v_cost.front_left_leg_to_platform = 0.1
    # v_cost.rear_right_leg_to_platform = 0.1
    # v_cost.rear_left_leg_to_platform = 0.1

    for n in range(N):
        # Joint limits
        prog.AddBoundingBoxConstraint(
            plant.GetPositionLowerLimits(),
            plant.GetPositionUpperLimits(),
            q[:, n],
        )
        # Joint velocity limits
        prog.AddBoundingBoxConstraint(
            plant.GetVelocityLowerLimits(),
            plant.GetVelocityUpperLimits(),
            v[:, n],
        )
        # Unit quaternions
        AddUnitQuaternionConstraintOnPlant(plant, q[:, n], prog)
        # Body orientation
        if n == 0:
            prog.AddConstraint(
                OrientationConstraint(
                    plant,
                    body_frame,
                    RotationMatrix(),
                    plant.world_frame(),
                    RotationMatrix(),
                    0.1,
                    context[n],
                ),
                q[:, n],
            )
        # Initial guess for all joint angles is the home position
        prog.SetInitialGuess(
            q[:, n], q0
        )  # Solvers get stuck if the quaternion is initialized with all zeros.

        # Running costs:
        prog.AddQuadraticErrorCost(np.diag(q_cost), q0, q[:, n])
        prog.AddQuadraticErrorCost(np.diag(v_cost), [0] * nv, v[:, n])

    # Make a new autodiff context for this constraint (to maximize cache hits)
    ad_velocity_dynamics_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def velocity_dynamics_constraint(vars, context_index):
        h, q, v, qn = np.split(vars, [1, 1 + nq, 1 + nq + nv])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                q,
                ad_plant.GetPositions(ad_velocity_dynamics_context[context_index]),
            ):
                ad_plant.SetPositions(ad_velocity_dynamics_context[context_index], q)
            v_from_qdot = ad_plant.MapQDotToVelocity(
                ad_velocity_dynamics_context[context_index], (qn - q) / h
            )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            v_from_qdot = plant.MapQDotToVelocity(context[context_index], (qn - q) / h)
        return v - v_from_qdot

    for n in range(N - 1):
        prog.AddConstraint(
            partial(velocity_dynamics_constraint, context_index=n),
            lb=[0] * nv,
            ub=[0] * nv,
            vars=np.concatenate(([h[n]], q[:, n], v[:, n], q[:, n + 1])),
        )

    # dyanmics of motion for the blades
    contact_force = [
        prog.NewContinuousVariables(3, N - 1, f"foot{foot}_contact_force")
        for foot in range(num_feet)
    ]

    # for i in range(4):
    #     for n in range(N-1):
    #         prog.AddLinearConstraint(contact_force[i][2, n] >= 1e-3)

    # Center of mass variables and constraints
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N - 1, "comddot")
    # Initial CoM x,y position == 0
    # prog.AddBoundingBoxConstraint(0, 0, com[:2, 0])
    # Initial CoM z vel == 0
    prog.AddBoundingBoxConstraint(0, 0, comdot[2, 0])
    # CoM height
    # prog.AddBoundingBoxConstraint(0.125, np.inf, com[2, :])
    # CoM x velocity >= 0
    prog.AddBoundingBoxConstraint(0, np.inf, comdot[0, :])
    prog.AddBoundingBoxConstraint(5, 5, com[:2, 0])
    # CoM final x position
    if is_laterally_symmetric:
        prog.AddBoundingBoxConstraint(
            stride_length / 2.0, stride_length / 2.0, com[0, -1]
        )
    else:
        prog.AddBoundingBoxConstraint(stride_length, stride_length, com[0, -1])

    #adding cost to optimize for velocity
    for n in range(N-1):
        prog.AddQuadraticCost(-3*comdot[:,n].T @ comdot[:,n])
    
    prog.AddQuadraticCost(-1*com[:, n].T @ com[:, n]) # optimizing on distance smooths out trajectory

    # CoM dynamics
    for n in range(N - 1):
        # Note: The original matlab implementation used backwards Euler (here and throughout),
        # which is a little more consistent with the LCP contact models.
        prog.AddConstraint(eq(com[:, n + 1], com[:, n] + h[n] * comdot[:, n]))
        prog.AddConstraint(eq(comdot[:, n + 1], comdot[:, n] + h[n] * comddot[:, n]))
        prog.AddConstraint(
            eq(
                total_mass * comddot[:, n],
                sum(contact_force[i][:, n] for i in range(4)) + total_mass * gravity,
            )
        )
    
    # Angular momentum (about the center of mass)
    H = prog.NewContinuousVariables(3, N, "H")
    Hdot = prog.NewContinuousVariables(3, N - 1, "Hdot")
    prog.SetInitialGuess(H, np.zeros((3, N)))
    prog.SetInitialGuess(Hdot, np.zeros((3, N - 1)))

    # Hdot = sum_i cross(p_FootiW-com, contact_force_i)
    def angular_momentum_constraint(vars, context_index):
        q, com, Hdot, contact_force = np.split(vars, [nq, nq + 3, nq + 6])
        contact_force = contact_force.reshape(3, 4, order="F")
        if isinstance(vars[0], AutoDiffXd):
            dq = ExtractGradient(q)
            q = ExtractValue(q)
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_feet):
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    #[0, 0, -0.4325],
                    [0, 0, -0.03],
                    plant.world_frame(),
                )
                Jq_WF = plant.CalcJacobianTranslationalVelocity(
                    context[context_index],
                    JacobianWrtVariable.kQDot,
                    foot_frame[i],
                    #[0, 0, -0.4325],
                    [0, 0, -0.03],
                    plant.world_frame(),
                    plant.world_frame(),
                )
                ad_p_WF = InitializeAutoDiff(p_WF, Jq_WF @ dq)
                torque = torque + np.cross(
                    ad_p_WF.reshape(3) - com, contact_force[:, i]
                )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_feet):
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    [0, 0, -0.03],
                    #[0, 0, -0.4325],
                    plant.world_frame(),
                )
                torque += np.cross(p_WF.reshape(3) - com, contact_force[:, i])
        return Hdot - torque

    # def TEST_CONSTRAINT(q, context_index):
    #     for foot in range(4):
    #         if in_stance[foot, context_index]:
    #             i = foot
    #             friction = [0.005, 0.95, 0] # axial should be small, lateral should be big! this is always along the foot

    #             # Calculate normal force (e.g., based on skater's weight)
    #             normal_force = in_stance[foot, context_index] * 4 * 9.81 * total_mass

    #             p_WF = plant.CalcPointsPositions(
    #                 context[context_index],
    #                 foot_frame[i],
    #                 [0, 0, -0.03],
    #                 #[0, 0, -0.4325],
    #                 plant.world_frame(),
    #             )

    #             leg_WF = plant.CalcPointsPositions(
    #                 context[context_index],
    #                 leg_frame[i],
    #                 [0, 0, -0.03365],
    #                 #[0, 0, -0.4325],
    #                 plant.world_frame(),
    #             )

    #             # foot_vector = p_WF
    #             # print(foot_vector)
    #             # print(-np.arctan(foot_vector[1], foot_vector[0]))
    #             # theta = np.arctan(foot_vector[1], foot_vector[0])[0]

    #             positions = plant.GetPositions(context[context_index])
    #             context_position = positions[foot_indices[i]]
    #             # ad_context = AutoDiffXd(context_position)
    #             # platform_position = q[foot_indices[i], context_index]
    #             # plant.SetPositions(context, q)
    #             # print("POSITIONS", positions)

    #             theta = context_position
    #             rotation_matrix = np.array([[np.cos(theta), np.sin(theta), 0],
    #                                         [-np.sin(theta), np.cos(theta), 0],
    #                                         [0, 0, 1]])
    #             friction_cone = rotation_matrix @ friction
            
    #             # Add friction cone constraints in foot frame
    #             mu_x = np.abs(friction_cone[0])
    #             print("mu_x", mu_x)
    #             mu_y = np.abs(friction_cone[1])
    #             print("mu_y", mu_y)

    #             prog.AddLinearConstraint(
    #                 contact_force[foot][0, context_index] <= mu_x  * contact_force[foot][2, context_index]
    #             )
    #             prog.AddLinearConstraint(
    #                 -contact_force[foot][0, context_index] <= mu_x * contact_force[foot][2, context_index]
    #             )
    #             prog.AddLinearConstraint(
    #                 contact_force[foot][1, context_index] <=  mu_y * contact_force[foot][2, context_index]
    #             )
    #             prog.AddLinearConstraint(
    #                 -contact_force[foot][1, context_index] <=  mu_y * contact_force[foot][2, context_index]
    #             )

    #             # symbolic_value = x_sym.Evaluate({x_sym: x_value})
    #             # symbolic_exp_x = mu_x
    #             # symbolic_exp_y = mu_y

    #             # def x_friction_constraint(contact_force):
    #             #     # contact_force_values = ExtractValue(contact_force)
    #             #     # force = contact_force_values.astype(np.float32)
    #             #     return force[0] - mu_x * force[2]

    #             # def y_friction_constraint(contact_force):
    #             #     # contact_force_values = ExtractValue(contact_force)
    #             #     # force = contact_force_values.astype(np.float32)
    #             #     return force[1] - mu_y * force[2]

    #             # prog.AddConstraint(
    #             #     x_friction_constraint,
    #             #     lb=[-np.inf],  # No lower bound
    #             #     ub=[0],     # Upper bound of 0
    #             #     vars=forces,
    #             #     # vars=contact_force[foot][:, context_index],
    #             # )

    #             # prog.AddConstraint(
    #             #     y_friction_constraint,
    #             #     lb=[-np.inf],  # No lower bound
    #             #     ub=[0],     # Upper bound of 0
    #             #     vars=forces,
    #             #     # vars=contact_force[foot][:, context_index],
    #             # )

    #             # normal force >=0, normal_force == 0 if not in_stance
    #             prog.AddBoundingBoxConstraint(
    #                 0,
    #                 in_stance[foot, context_index] * 4 * 9.81 * total_mass,
    #                 contact_force[foot][2, context_index],
    #             )

    def friction_constraint(vars, context_index):
        #for n in range(N - 1):
        # q, com, Hdot, contact_force = np.split(vars, [nq, nq + 3, nq + 6])
        qv, com, H = np.split(vars, [nq + nv, nq + nv + 3])
        for foot in range(num_feet):
            if in_stance[foot, context_index]:
                if not np.array_equal(q, plant.GetPositionsAndVelocities(context[context_index])):
                    plant.SetPositionsAndVelocities(context[context_index], qv)

                p_WF = plant.CalcPointsPositions(
                        context[context_index],
                        foot_frame[i],
                        [0, 0, -0.03],
                        plant.world_frame()
                    )

                leg_WF = plant.CalcPointsPositions(
                        context[context_index],
                        leg_frame[i],
                        [0, 0, -0.03365],
                        plant.world_frame()
                )

                #     plant.world_frame(),
                # )
                # # print(X_BW)

                # R_BW = X_BW.rotation()

                # Calculate normal force (e.g., based on skater's weight)
                normal_force = in_stance[foot, context_index] * 4 * 9.81 * total_mass

                # Transform friction cone directions to the foot frame
                wheel_angle = np.arctan(p_WF[1], p_WF[0])[0]
                # print(wheel_angle)

                rotation_matrix = np.array([[np.cos(wheel_angle), np.sin(wheel_angle), 0], 
                                            [-np.sin(wheel_angle), np.cos(wheel_angle), 0], 
                                            [0, 0, 1]])
                friction_cone_x = axial_friction @ np.linalg.inv(rotation_matrix) 
                friction_cone_y = lateral_friction @ np.linalg.inv(rotation_matrix) 
                # friction_cone_x_foot = np.dot(RotationMatrix(), friction_cone_local_x)
                # friction_cone_y_foot = np.dot(RotationMatrix(), friction_cone_local_y)
                # print(friction_cone_x[0])

                # Add friction cone constraints in foot frame
                # mu_x = np.abs(friction_cone_x[0] + friction_cone_y[0])
                # mu_y = np.abs(friction_cone_y[1] + + friction_cone_x[1])

                # prog.AddLinearConstraint(
                #     contact_force[foot][0, context_index] <= mu_x  * contact_force[foot][2, n]
                # )
                # prog.AddLinearConstraint(
                #     -contact_force[foot][0, context_index] <= mu_x * contact_force[foot][2, n]
                # )
                # prog.AddLinearConstraint(
                #     contact_force[foot][1, context_index] <=  mu_y * contact_force[foot][2, n]
                # )
                # prog.AddLinearConstraint(
                #     -contact_force[foot][1, context_index] <=  mu_y * contact_force[foot][2, n]
                # )
                # # normal force >=0, normal_force == 0 if not in_stance
                prog.AddBoundingBoxConstraint(
                    0,
                    in_stance[foot, n] * 4 * 9.81 * total_mass,
                    contact_force[foot][2, context_index],
                )

    axial_mu = 0.005
    lateral_mu = 0.95

    for n in range(N - 1):
        prog.AddConstraint(eq(H[:, n + 1], H[:, n] + h[n] * Hdot[:, n]))
        Fn = np.concatenate([contact_force[i][:, n] for i in range(4)])
        prog.AddConstraint(
            partial(angular_momentum_constraint, context_index=n),
            lb=np.zeros(3),
            ub=np.zeros(3),
            vars=np.concatenate((q[:, n], com[:, n], Hdot[:, n], Fn)),
        )

        prog.AddConstraint(
            partial(friction_constraint, context_index=n),
            lb = np.array([axial_mu, axial_mu, 0]),
            ub = np.array([lateral_mu, lateral_mu, 0])
            vars = np.concatenate((q[:, n], v[:, n], com[:, n], H[:, n]))
        )
        # TEST_CONSTRAINT(np.concatenate((q[:, n], com[:, n], Hdot[:, n], Fn)), n)
    

    # com == CenterOfMass(q), H = SpatialMomentumInWorldAboutPoint(q, v, com)
    # Make a new autodiff context for this constraint (to maximize cache hits)
    com_constraint_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def com_constraint(vars, context_index):
        qv, com, H = np.split(vars, [nq + nv, nq + nv + 3])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                qv,
                ad_plant.GetPositionsAndVelocities(com_constraint_context[context_index])
            ):
                ad_plant.SetPositionsAndVelocities(
                    com_constraint_context[context_index], qv
                )
            com_q = ad_plant.CalcCenterOfMassPositionInWorld(
                com_constraint_context[context_index], [spot]
            )
            H_qv = ad_plant.CalcSpatialMomentumInWorldAboutPoint(
                com_constraint_context[context_index], [spot], com
            ).rotational()
        else:
            if not np.array_equal(
                qv, plant.GetPositionsAndVelocities(context[context_index])
            ):
                plant.SetPositionsAndVelocities(context[context_index], qv)
            com_q = plant.CalcCenterOfMassPositionInWorld(
                context[context_index], [spot]
            )
            H_qv = plant.CalcSpatialMomentumInWorldAboutPoint(
                context[context_index], [spot], com
            ).rotational()
        return np.concatenate((com_q - com, H_qv - H))

    for n in range(N):
        prog.AddConstraint(
            partial(com_constraint, context_index=n),
            lb=np.zeros(6),
            ub=np.zeros(6),
            vars=np.concatenate((q[:, n], v[:, n], com[:, n], H[:, n])),
        )

    # TODO: Add collision constraints
    # Wheels cannot collide:

    # Kinematic constraints
    def fixed_position_constraint(vars, context_index, frame):
        q, qn = np.split(vars, [nq])
        if not np.array_equal(q, plant.GetPositions(context[context_index])):
            plant.SetPositions(context[context_index], q)
        if not np.array_equal(qn, plant.GetPositions(context[context_index + 1])):
            plant.SetPositions(context[context_index + 1], qn)
        p_WF = plant.CalcPointsPositions(
            context[context_index], frame, [0, 0, 0], plant.world_frame()
        )
        p_WF_n = plant.CalcPointsPositions(
            context[context_index + 1], frame, [0, 0, 0], plant.world_frame()
        )
        if isinstance(vars[0], AutoDiffXd):
            J_WF = plant.CalcJacobianTranslationalVelocity(
                context[context_index],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            J_WF_n = plant.CalcJacobianTranslationalVelocity(
                context[context_index + 1],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            return InitializeAutoDiff(
                p_WF_n - p_WF,
                J_WF_n @ ExtractGradient(qn) - J_WF @ ExtractGradient(q),
            )
        else:
            return p_WF_n - p_WF

    for i in range(num_feet):
        for n in range(N):
            if in_stance[i, n]:
                # foot should be on the ground (world position z=0)
                if i == 0 or i == 2:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            # [np.inf, np.inf, 0],
                            [-np.inf, -np.inf, 0],
                            [np.inf, -0.01, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    #SECOND WHEEL TRIAL RUN
                    prog.AddConstraint( #added
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            [-np.inf, -np.inf, 0],
                            # [np.inf, np.inf, 0],
                            [np.inf, -0.01, 0],
                            second_foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                else:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            # [np.inf, np.inf, 0],
                            [-np.inf, 0.01, 0],
                            [np.inf, np.inf, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    prog.AddConstraint( #added
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            [-np.inf, 0.01, 0],
                            # [np.inf, np.inf, 0],
                            [np.inf, np.inf, 0],
                            second_foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )

    def AddAntiSymmetricPair(a, b):
            prog.AddLinearEqualityConstraint(a[0] == -b[-1])
            prog.AddLinearEqualityConstraint(a[-1] == -b[0])

    def AddSymmetricPair(a, b):
        prog.AddLinearEqualityConstraint(a[0] == b[-1])
        prog.AddLinearEqualityConstraint(a[-1] == b[0])

    # Periodicity constraints
    if is_laterally_symmetric:
        # Joints
        AddAntiSymmetricPair(q_view.front_left_hip_x, q_view.front_right_hip_x)
        AddSymmetricPair(q_view.front_left_hip_y, q_view.front_right_hip_y)
        AddSymmetricPair(q_view.front_left_knee, q_view.front_right_knee)
        AddAntiSymmetricPair(q_view.rear_left_hip_x, q_view.rear_right_hip_x)
        AddSymmetricPair(q_view.rear_left_hip_y, q_view.rear_right_hip_y)
        AddSymmetricPair(q_view.rear_left_knee, q_view.rear_right_knee)
        # prog.AddLinearEqualityConstraint(q_view.body_y[0] == -q_view.body_y[-1])
        # prog.AddLinearEqualityConstraint(q_view.body_z[0] == q_view.body_z[-1])
        # Body orientation must be in the xz plane:
        prog.AddBoundingBoxConstraint(0, 0, q_view.body_qx[[0, -1]])
        prog.AddBoundingBoxConstraint(0, 0, q_view.body_qz[[0, -1]])

        # Floating base velocity
        # prog.AddLinearEqualityConstraint(v_view.body_vx[0] == v_view.body_vx[-1])
        # prog.AddLinearEqualityConstraint(v_view.body_vy[0] == -v_view.body_vy[-1])
        # prog.AddLinearEqualityConstraint(v_view.body_vz[0] == v_view.body_vz[-1])

        # # CoM velocity
        # prog.AddLinearEqualityConstraint(comdot[0, 0] == comdot[0, -1])
        # prog.AddLinearEqualityConstraint(comdot[1, 0] == -comdot[1, -1])
        # prog.AddLinearEqualityConstraint(comdot[2, 0] == comdot[2, -1])
    if is_wheel_symmetric:
        AddAntiSymmetricPair(q_view.rear_left_leg_to_platform, q_view.rear_right_leg_to_platform)
        AddAntiSymmetricPair(q_view.front_left_leg_to_platform, q_view.front_right_leg_to_platform)
    else:
        # Everything except body_x is periodic
        q_selector = PositionView([True] * nq)
        q_selector.body_x = False
        prog.AddLinearConstraint(eq(q[q_selector, 0], q[q_selector, -1]))
        prog.AddLinearConstraint(eq(v[:, 0], v[:, -1]))

    # TODO: Set solver parameters (mostly to make the worst case solve times less bad)
    snopt = SnoptSolver().solver_id()
    prog.SetSolverOption(snopt, "Iterations Limits", 1e5 if running_as_notebook else 1)
    prog.SetSolverOption(
        snopt, "Major Iterations Limit", 200 if running_as_notebook else 1
    )
    prog.SetSolverOption(snopt, "Major Feasibility Tolerance", 5e-6)
    prog.SetSolverOption(snopt, "Major Optimality Tolerance", 1e-4)
    prog.SetSolverOption(snopt, "Superbasics limit", 2000)
    prog.SetSolverOption(snopt, "Linesearch tolerance", 0.9)
    # prog.SetSolverOption(snopt, 'Print file', 'snopt.out')

    result = Solve(prog)
    print(result.get_solver_id().name())
    print(result.is_success())  # We expect this to be false if iterations are limited.

    def HalfStrideToFullStride(a):
        b = PositionView(np.copy(a))

        b.body_y = -a.body_y
        # Mirror quaternion so that roll=-roll, yaw=-yaw
        b.body_qx = -a.body_qx
        b.body_qz = -a.body_qz

        b.front_left_hip_x = -a.front_right_hip_x
        b.front_right_hip_x = -a.front_left_hip_x
        b.rear_left_hip_x = -a.rear_right_hip_x
        b.rear_right_hip_x = -a.rear_left_hip_x

        b.front_left_hip_y = a.front_right_hip_y
        b.front_right_hip_y = a.front_left_hip_y
        b.rear_left_hip_y = a.rear_right_hip_y
        b.rear_right_hip_y = a.rear_left_hip_y

        b.front_left_knee = a.front_right_knee
        b.front_right_knee = a.front_left_knee
        b.rear_left_knee = a.rear_right_knee
        b.rear_right_knee = a.rear_left_knee

        return b

    # Animate trajectory
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    t_sol = np.cumsum(np.concatenate(([0], result.GetSolution(h))))
    q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
    visualizer.StartRecording()
    num_strides = 4
    t0 = t_sol[0]
    tf = t_sol[-1]
    T = tf * num_strides * (2.0 if is_laterally_symmetric else 1.0)
    for t in np.hstack((np.arange(t0, T, 1.0 / 32.0), T)):
        context.SetTime(t)
        stride = (t - t0) // (tf - t0)
        ts = (t - t0) % (tf - t0)
        qt = PositionView(q_sol.value(ts))
        if is_laterally_symmetric:
            if stride % 2 == 1:
                qt = HalfStrideToFullStride(qt)
                qt.body_x += stride_length / 2.0
            stride = stride // 2
        qt.body_x += stride * stride_length
        plant.SetPositions(plant_context, qt[:])
        diagram.ForcedPublish(context)

    visualizer.StopRecording()
    visualizer.PublishRecording()


gait_optimization("swizzle")


In [ ]:
# FORWARD GAIT WITH MODIFIED FRICTION MODEL
# attempted to use collision geometries and contact results in Drake in order to calculate friction constraints
# was not able to successfully run the simulation, but would attempt to develop with more fidelity if there was more time on the project


def autoDiffArrayEqual(a, b):
    return np.array_equal(a, b) and np.array_equal(
        ExtractGradient(a), ExtractGradient(b)
    )

def gait_optimization(gait="walking_trot", initial_pos=None):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    ConfigureParser(parser)
    (spot,) = parser.AddModels("spotblades.dmd.yaml")  # changed to spotblades
    plant.Finalize()

    # # plant.RegisterAsSourceForSceneGraph(scene_graph)
    # builder.Connect(
    #     scene_graph.get_query_output_port(),
    #     plant.get_geometry_query_input_port(),
    # )

    if not plant.geometry_source_is_registered():
        plant.RegisterAsSourceForSceneGraph(scene_graph)
        print("registered")

    ConnectContactResultsToDrakeVisualizer(builder, plant, scene_graph)
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(root_context=context)
    diagram.ForcedPublish(context)

    # def trace_to_output(diagram_or_builder, input_port):
    #     system = input_port.get_system()
    #     input_locator = (system, input_port.get_index())
    #     connection_map = diagram_or_builder.connection_map()
    #     output_system, output_index = connection_map[input_locator]
    #     output_port = output_system.get_output_port(output_index)
    #     return output_port
    
    # print(trace_to_output(diagram, plant.get_geometry_query_input_port()))
    # print(trace_to_output(diagram, plant.get_geometry_query_input_port()).get_system())
    # # print(trace_to_output(diagram, plant.get_geometry_query_input_port()).get_connected_to()[0])
    # print(trace_to_output(diagram, plant.get_geometry_query_input_port()).Eval())
    # print(scene_graph.get_query_output_port().get_name())
    # print(scene_graph.get_query_output_port().get_system())
    # # print(scene_graph.get_query_output_port().get_connected_to()[0])
    # print(scene_graph.get_query_output_port().Eval())

    q0 = plant.GetPositions(plant_context)
    body_frame = plant.GetFrameByName("body")

    PositionView = namedview(
        "Positions", plant.GetPositionNames(spot, always_add_suffix=False)
    )
    VelocityView = namedview(
        "Velocities",
        plant.GetVelocityNames(spot, always_add_suffix=False),
    )

    total_mass = plant.CalcTotalMass(plant_context, [spot])
    gravity = plant.gravity_field().gravity_vector()

    foot_frame = [
        plant.GetFrameByName("first_front_right_wheel"),
        plant.GetFrameByName("first_front_left_wheel"),
        plant.GetFrameByName("first_rear_right_wheel"),
        plant.GetFrameByName("first_rear_left_wheel"),
        plant.GetFrameByName("second_front_right_wheel"),
        plant.GetFrameByName("second_front_left_wheel"),
        plant.GetFrameByName("second_rear_right_wheel"),
        plant.GetFrameByName("second_rear_left_wheel")
    ]

    num_feet = len(foot_frame)

    lateral_friction = CoulombFriction(1.0, 0.9)
    axial_friction = CoulombFriction(0.05, 0.03)

    lateral_friction_property = ProximityProperties()
    lateral_friction_property.AddProperty("material", "coulomb_friction", lateral_friction)

    axial_friction_property = ProximityProperties()
    axial_friction_property.AddProperty("material", "coulomb_friction", axial_friction)

    for foot in foot_frame:
        frame_id = plant.GetBodyFrameIdOrThrow(foot.body().index())
        source_id = plant.get_source_id()

        # Create a new GeometryInstance for each foot 
        lateral_friction_geometry = GeometryInstance(
            RigidTransform(), Box(0.01, 0.035, 0.035), "lateral_friction_collision"
        )
        lateral_friction_id = scene_graph.RegisterGeometry(source_id, frame_id, lateral_friction_geometry)
        scene_graph.AssignRole(source_id, lateral_friction_id, lateral_friction_property)

        # Create a new GeometryInstance for each foot
        axial_friction_geometry = GeometryInstance(
            RigidTransform(), Box(0.01, 0.035, 0.035), "axial_friction_collision"
        )
        axial_friction_id = scene_graph.RegisterGeometry(source_id, frame_id, axial_friction_geometry)
        scene_graph.AssignRole(source_id, axial_friction_id, axial_friction_property)

    # Rest of the gait_optimization function...
    # (your original code continues here)
    # setup gait
    is_laterally_symmetric = False
    check_self_collision = True
    if gait == "skate":
        N = 21
        in_stance = np.zeros((num_feet, N))
        in_stance[0, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[1, 0:41] = 1
        in_stance[2, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[3, 0:18] = 1 #only right leg lifted for 26:N in stance
        in_stance[4, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[5, 0:41] = 1
        in_stance[6, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[7, 0:18] = 1 #only right leg lifted for 26:N in stance
        # in_stance[2, 0:41] = 1 
        # in_stance[3, 0:41] = 1
        # can't cross in front of torso
        speed = 0.7
        is_laterally_symmetric = True
        check_self_collision = True
        stride_length = 0.55
    elif gait == "swizzle":
        N = 21
        in_stance = np.zeros((num_feet, N))
        in_stance[0, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[1, 0:41] = 1
        in_stance[2, 0:41] = 1 
        in_stance[3, 0:41] = 1
        in_stance[4, 0:41] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[5, 0:41] = 1
        in_stance[6, 0:41] = 1 
        in_stance[7, 0:41] = 1
        # can't cross in front of torso
        speed = 0.7
        is_laterally_symmetric = False
        is_wheel_symmetric = False
        check_self_collision = True
        stride_length = 0.55
    elif gait == "stop":
        N = 21
        in_stance = np.ones((num_feet, N))
        speed = 0.02
        is_laterally_symmetric = True
        check_self_collision = True
        stride_length = 0.3
    else:
        raise RuntimeError("Unknown gait.")

    T = stride_length / speed
    if is_laterally_symmetric:
        T = T / 2.0

    prog = MathematicalProgram()

    # Time steps
    h = prog.NewContinuousVariables(N - 1, "h")
    prog.AddBoundingBoxConstraint(0.5 * T / N, 2.0 * T / N, h) 
    prog.AddLinearConstraint(sum(h) >= 0.9 * T)
    prog.AddLinearConstraint(sum(h) <= 1.1 * T)

    # Create one context per time step (to maximize cache hits)
    # context = [plant.CreateDefaultContext() for i in range(N)]
    context = [plant_context for i in range(N)]
    # We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
    ad_plant = plant.ToAutoDiffXd()

    # Joint positions and velocities
    nq = plant.num_positions()
    nv = plant.num_velocities()
    q = prog.NewContinuousVariables(nq, N, "q")
    v = prog.NewContinuousVariables(nv, N, "v")
    q_view = PositionView(q)
    v_view = VelocityView(v)
    q0_view = PositionView(q0)
    # Joint costs
    # print(q_cost)
    # print(v_view)
    q_cost = PositionView([1] * nq)
    print(q_cost)
    v_cost = VelocityView([1] * nv)
    q_cost.body_x = 0
    q_cost.body_y = 0
    q_cost.body_qx = 0
    q_cost.body_qy = 0
    q_cost.body_qz = 0
    q_cost.body_qw = 0
    
    q_cost.front_left_hip_x = 0.2
    q_cost.front_right_hip_x = 0.2
    q_cost.rear_left_hip_x = 0.2
    q_cost.rear_right_hip_x = 0.2
    v_cost.body_vx = 0
    v_cost.body_wx = 0
    v_cost.body_wy = 0
    v_cost.body_wz = 0

    q_cost.front_right_leg_to_wheel = 0.1
    q_cost.second_front_right_leg_to_wheel = 0.1
    q_cost.front_left_leg_to_wheel = 0.1
    q_cost.second_front_left_leg_to_wheel = 0.1
    # q_cost.rear_right_leg_to_wheel = 0.1
    # q_cost.second_rear_right_leg_to_wheel = 0.1
    # q_cost.rear_right_leg_to_wheel = 0.1
    # q_cost.second_rear_left_leg_to_wheel = 0.1
    # q_cost.front_right_leg_to_platform = 0.1
    # q_cost.front_left_leg_to_platform = 0.1
    # q_cost.rear_right_leg_to_platform = 0.1
    # q_cost.rear_left_leg_to_platform = 0.1

    # v_cost.front_right_leg_to_wheel = 0.1
    # v_cost.second_front_right_leg_to_wheel = 0.1
    # v_cost.front_left_leg_to_wheel = 0.1
    # v_cost.second_front_left_leg_to_wheel = 0.1
    # v_cost.rear_right_leg_to_wheel = 0.1
    # v_cost.second_rear_right_leg_to_wheel = 0.1
    # v_cost.rear_right_leg_to_wheel = 0.1
    # v_cost.second_rear_left_leg_to_wheel = 0.1
    # v_cost.front_right_leg_to_platform = 0.1
    # v_cost.front_left_leg_to_platform = 0.1
    # v_cost.rear_right_leg_to_platform = 0.1
    # v_cost.rear_left_leg_to_platform = 0.1

    for n in range(N):
        # Joint limits
        prog.AddBoundingBoxConstraint(
            plant.GetPositionLowerLimits(),
            plant.GetPositionUpperLimits(),
            q[:, n],
        )
        # Joint velocity limits
        prog.AddBoundingBoxConstraint(
            plant.GetVelocityLowerLimits(),
            plant.GetVelocityUpperLimits(),
            v[:, n],
        )
        # Unit quaternions
        AddUnitQuaternionConstraintOnPlant(plant, q[:, n], prog)
        # Body orientation
        if n == 0:
            prog.AddConstraint(
                OrientationConstraint(
                    plant,
                    body_frame,
                    RotationMatrix(),
                    plant.world_frame(),
                    RotationMatrix(),
                    0.1,
                    context[n],
                ),
                q[:, n],
            )
        # Initial guess for all joint angles is the home position
        prog.SetInitialGuess(
            q[:, n], q0
        )  # Solvers get stuck if the quaternion is initialized with all zeros.

        # Running costs:
        prog.AddQuadraticErrorCost(np.diag(q_cost), q0, q[:, n])
        prog.AddQuadraticErrorCost(np.diag(v_cost), [0] * nv, v[:, n])

    # Make a new autodiff context for this constraint (to maximize cache hits)
    ad_velocity_dynamics_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def velocity_dynamics_constraint(vars, context_index):
        h, q, v, qn = np.split(vars, [1, 1 + nq, 1 + nq + nv])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                q,
                ad_plant.GetPositions(ad_velocity_dynamics_context[context_index]),
            ):
                ad_plant.SetPositions(ad_velocity_dynamics_context[context_index], q)
            v_from_qdot = ad_plant.MapQDotToVelocity(
                ad_velocity_dynamics_context[context_index], (qn - q) / h
            )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            v_from_qdot = plant.MapQDotToVelocity(context[context_index], (qn - q) / h)
        return v - v_from_qdot

    for n in range(N - 1):
        prog.AddConstraint(
            partial(velocity_dynamics_constraint, context_index=n),
            lb=[0] * nv,
            ub=[0] * nv,
            vars=np.concatenate(([h[n]], q[:, n], v[:, n], q[:, n + 1])),
        )

    # Center of mass variables and constraints
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N - 1, "comddot")
    # Initial CoM x,y position == 0
    # prog.AddBoundingBoxConstraint(0, 0, com[:2, 0])
    # Initial CoM z vel == 0
    prog.AddBoundingBoxConstraint(0, 0, comdot[2, 0])
    # CoM height
    # prog.AddBoundingBoxConstraint(0.125, np.inf, com[2, :])
    # CoM x velocity >= 0
    prog.AddBoundingBoxConstraint(0, np.inf, comdot[0, :])
    prog.AddBoundingBoxConstraint(5, 5, com[:2, 0])
    # CoM final x position
    if is_laterally_symmetric:
        prog.AddBoundingBoxConstraint(
            stride_length / 2.0, stride_length / 2.0, com[0, -1]
        )
    else:
        prog.AddBoundingBoxConstraint(stride_length, stride_length, com[0, -1])

    #adding cost to optimize for velocity
    for n in range(N-1):
        prog.AddQuadraticCost(-3*comdot[:,n].T @ comdot[:,n])
    
    prog.AddQuadraticCost(-1*com[:, n].T @ com[:, n]) # optimizing on distance smooths out trajectory

    # dyanmics of motion for the blades
    contact_force = [
        prog.NewContinuousVariables(3, N - 1, f"foot{foot}_contact_force")
        for foot in range(num_feet)
    ]

    # CoM dynamics
    for n in range(N - 1):
        # Note: The original matlab implementation used backwards Euler (here and throughout),
        # which is a little more consistent with the LCP contact models.
        prog.AddConstraint(eq(com[:, n + 1], com[:, n] + h[n] * comdot[:, n]))
        prog.AddConstraint(eq(comdot[:, n + 1], comdot[:, n] + h[n] * comddot[:, n]))
        prog.AddConstraint(
            eq(
                total_mass * comddot[:, n],
                sum(contact_force[i][:, n] for i in range(4)) + total_mass * gravity,
            )
        )
    
    # Angular momentum (about the center of mass)
    H = prog.NewContinuousVariables(3, N, "H")
    Hdot = prog.NewContinuousVariables(3, N - 1, "Hdot")
    prog.SetInitialGuess(H, np.zeros((3, N)))
    prog.SetInitialGuess(Hdot, np.zeros((3, N - 1)))

    # Hdot = sum_i cross(p_FootiW-com, contact_force_i)
    def angular_momentum_constraint(vars, context_index):
        q, com, Hdot, contact_force = np.split(vars, [nq, nq + 3, nq + 6])
        contact_force = contact_force.reshape(3, 4, order="F")
        if isinstance(vars[0], AutoDiffXd):
            dq = ExtractGradient(q)
            q = ExtractValue(q)
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(4):
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    #[0, 0, -0.4325],
                    [0, 0, -0.03],
                    plant.world_frame(),
                )
                Jq_WF = plant.CalcJacobianTranslationalVelocity(
                    context[context_index],
                    JacobianWrtVariable.kQDot,
                    foot_frame[i],
                    #[0, 0, -0.4325],
                    [0, 0, -0.03],
                    plant.world_frame(),
                    plant.world_frame(),
                )
                ad_p_WF = InitializeAutoDiff(p_WF, Jq_WF @ dq)
                torque = torque + np.cross(
                    ad_p_WF.reshape(3) - com, contact_force[:, i]
                )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_feet):
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    [0, 0, -0.03],
                    #[0, 0, -0.4325],
                    plant.world_frame(),
                )
                torque += np.cross(p_WF.reshape(3) - com, contact_force[:, i])
        return Hdot - torque

    ground_frame = plant.GetFrameByName("ground")
    ground_index = ground_frame.index()

    def friction_constraint(vars, context_index):
        qv, com, H = np.split(vars, [nq + nv, nq + nv + 3])
        print(context_index)
        if not np.array_equal(qv, plant.GetPositionsAndVelocities(context[context_index])):
            plant.SetPositionsAndVelocities(context[context_index], qv)

        contact_results = plant.get_contact_results_output_port().Eval(context[context_index])
        constraints = []

        for foot in range(num_feet):
            if in_stance[foot, context_index]:
                if foot >= len(contact_results.point_pair_contact_info()):
                    continue

                contact_pairs = 
                contact_point = contact_info.contact_point()
                foot_v = plant.CalcPointsSpatialVelocityInWorld(context[context_index], foot_frame[foot], contact_point)

                # if np.abs(foot_v[0]) > np.abs(foot_v[1]):
                #     friction = axial_friction
                # else:
                #     friction = lateral_friction
                
                normal_force = 4 * 9.81 * total_mass

                foot_v_mag = np.linalg.norm(foot_v)
                if foot_v_mag < 0.01:
                    mu_x = axial_friction.static_friction
                    mu_y = lateral_friction.static_friction
                else:
                    mu_x = axial_friction.dynamic_friction
                    mu_y = lateral_friction.dynamic_friction

                # constraints.append(contact_force[foot][0, context_index] - mu * normal_force)
                fx = -mu_x * normal_force * foot_v[0] / (foot_v_mag + 1e-6)
                fy = -mu_y * normal_force * foot_v[1] / (foot_v_mag + 1e-6)
                constraints.append(contact_force[foot][0, context_index]-fx)
                constraints.append(contact_force[foot][1, context_index]-fy)
            else:
                constraints.append(contact_force[foot][0, context_index])
                constraints.append(contact_force[foot][1, context_index])

        return np.array(constraints)
    
    # print(len(context))

    for n in range(N-1):
        prog.AddConstraint(eq(H[:, n + 1], H[:, n] + h[n] * Hdot[:, n]))
        Fn = np.concatenate([contact_force[i][:, n] for i in range(4)])
        prog.AddConstraint(
            partial(angular_momentum_constraint, context_index=n),
            lb=np.zeros(3),
            ub=np.zeros(3),
            vars=np.concatenate((q[:, n], com[:, n], Hdot[:, n], Fn)),
        )
        prog.AddConstraint(
            partial(friction_constraint, context_index=n),
            lb = np.zeros(num_feet*2),
            ub = np.zeros(num_feet*2),
            vars = np.concatenate((q[:, n], v[:, n], com[:, n], H[:, n]))
        )
        for foot in range(num_feet):
            prog.AddBoundingBoxConstraint(
                0,
                in_stance[foot, n] * 4 * 9.81 * total_mass,
                contact_force[foot][2, n],
            )

    # com == CenterOfMass(q), H = SpatialMomentumInWorldAboutPoint(q, v, com)
    # Make a new autodiff context for this constraint (to maximize cache hits)
    com_constraint_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def com_constraint(vars, context_index):
        qv, com, H = np.split(vars, [nq + nv, nq + nv + 3])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                qv,
                ad_plant.GetPositionsAndVelocities(com_constraint_context[context_index])
            ):
                ad_plant.SetPositionsAndVelocities(
                    com_constraint_context[context_index], qv
                )
            com_q = ad_plant.CalcCenterOfMassPositionInWorld(
                com_constraint_context[context_index], [spot]
            )
            H_qv = ad_plant.CalcSpatialMomentumInWorldAboutPoint(
                com_constraint_context[context_index], [spot], com
            ).rotational()
        else:
            if not np.array_equal(
                qv, plant.GetPositionsAndVelocities(context[context_index])
            ):
                plant.SetPositionsAndVelocities(context[context_index], qv)
            com_q = plant.CalcCenterOfMassPositionInWorld(
                context[context_index], [spot]
            )
            H_qv = plant.CalcSpatialMomentumInWorldAboutPoint(
                context[context_index], [spot], com
            ).rotational()
        return np.concatenate((com_q - com, H_qv - H))

    for n in range(N):
        prog.AddConstraint(
            partial(com_constraint, context_index=n),
            lb=np.zeros(6),
            ub=np.zeros(6),
            vars=np.concatenate((q[:, n], v[:, n], com[:, n], H[:, n])),
        )

    # TODO: Add collision constraints
    # Wheels cannot collide:

    # Kinematic constraints
    def fixed_position_constraint(vars, context_index, frame):
        q, qn = np.split(vars, [nq])
        if not np.array_equal(q, plant.GetPositions(context[context_index])):
            plant.SetPositions(context[context_index], q)
        if not np.array_equal(qn, plant.GetPositions(context[context_index + 1])):
            plant.SetPositions(context[context_index + 1], qn)
        p_WF = plant.CalcPointsPositions(
            context[context_index], frame, [0, 0, 0], plant.world_frame()
        )
        p_WF_n = plant.CalcPointsPositions(
            context[context_index + 1], frame, [0, 0, 0], plant.world_frame()
        )
        if isinstance(vars[0], AutoDiffXd):
            J_WF = plant.CalcJacobianTranslationalVelocity(
                context[context_index],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            J_WF_n = plant.CalcJacobianTranslationalVelocity(
                context[context_index + 1],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            return InitializeAutoDiff(
                p_WF_n - p_WF,
                J_WF_n @ ExtractGradient(qn) - J_WF @ ExtractGradient(q),
            )
        else:
            return p_WF_n - p_WF

    for i in range(num_feet):
        for n in range(N):
            if in_stance[i, n]:
                # foot should be on the ground (world position z=0)
                if i == 0 or i == 2:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            # [np.inf, np.inf, 0],
                            [-np.inf, -np.inf, 0],
                            [np.inf, -0.01, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    # #SECOND WHEEL TRIAL RUN
                    # prog.AddConstraint( #added
                    #     PositionConstraint(
                    #         plant,
                    #         plant.world_frame(),
                    #         [-np.inf, -np.inf, 0],
                    #         # [np.inf, np.inf, 0],
                    #         [np.inf, -0.01, 0],
                    #         second_foot_frame[i],
                    #         #[0, 0, -0.4325],
                    #         [0, 0, -0.03],
                    #         context[n]),
                    #     q[:, n],
                    # )
                else:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            # [np.inf, np.inf, 0],
                            [-np.inf, 0.01, 0],
                            [np.inf, np.inf, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    # prog.AddConstraint( #added
                    #     PositionConstraint(
                    #         plant,
                    #         plant.world_frame(),
                    #         [-np.inf, 0.01, 0],
                    #         # [np.inf, np.inf, 0],
                    #         [np.inf, np.inf, 0],
                    #         second_foot_frame[i],
                    #         #[0, 0, -0.4325],
                    #         [0, 0, -0.03],
                    #         context[n]),
                    #     q[:, n],
                    # )

    def AddAntiSymmetricPair(a, b):
            prog.AddLinearEqualityConstraint(a[0] == -b[-1])
            prog.AddLinearEqualityConstraint(a[-1] == -b[0])

    def AddSymmetricPair(a, b):
        prog.AddLinearEqualityConstraint(a[0] == b[-1])
        prog.AddLinearEqualityConstraint(a[-1] == b[0])

    # Periodicity constraints
    if is_laterally_symmetric:
        # Joints
        AddAntiSymmetricPair(q_view.front_left_hip_x, q_view.front_right_hip_x)
        AddSymmetricPair(q_view.front_left_hip_y, q_view.front_right_hip_y)
        AddSymmetricPair(q_view.front_left_knee, q_view.front_right_knee)
        AddAntiSymmetricPair(q_view.rear_left_hip_x, q_view.rear_right_hip_x)
        AddSymmetricPair(q_view.rear_left_hip_y, q_view.rear_right_hip_y)
        AddSymmetricPair(q_view.rear_left_knee, q_view.rear_right_knee)
        # prog.AddLinearEqualityConstraint(q_view.body_y[0] == -q_view.body_y[-1])
        # prog.AddLinearEqualityConstraint(q_view.body_z[0] == q_view.body_z[-1])
        # Body orientation must be in the xz plane:
        prog.AddBoundingBoxConstraint(0, 0, q_view.body_qx[[0, -1]])
        prog.AddBoundingBoxConstraint(0, 0, q_view.body_qz[[0, -1]])

        # Floating base velocity
        # prog.AddLinearEqualityConstraint(v_view.body_vx[0] == v_view.body_vx[-1])
        # prog.AddLinearEqualityConstraint(v_view.body_vy[0] == -v_view.body_vy[-1])
        # prog.AddLinearEqualityConstraint(v_view.body_vz[0] == v_view.body_vz[-1])

        # # CoM velocity
        # prog.AddLinearEqualityConstraint(comdot[0, 0] == comdot[0, -1])
        # prog.AddLinearEqualityConstraint(comdot[1, 0] == -comdot[1, -1])
        # prog.AddLinearEqualityConstraint(comdot[2, 0] == comdot[2, -1])
    if is_wheel_symmetric:
        AddAntiSymmetricPair(q_view.rear_left_leg_to_platform, q_view.rear_right_leg_to_platform)
        AddAntiSymmetricPair(q_view.front_left_leg_to_platform, q_view.front_right_leg_to_platform)
    else:
        # Everything except body_x is periodic
        q_selector = PositionView([True] * nq)
        q_selector.body_x = False
        prog.AddLinearConstraint(eq(q[q_selector, 0], q[q_selector, -1]))
        prog.AddLinearConstraint(eq(v[:, 0], v[:, -1]))

    # TODO: Set solver parameters (mostly to make the worst case solve times less bad)
    snopt = SnoptSolver().solver_id()
    prog.SetSolverOption(snopt, "Iterations Limits", 1e5 if running_as_notebook else 1)
    prog.SetSolverOption(
        snopt, "Major Iterations Limit", 200 if running_as_notebook else 1
    )
    prog.SetSolverOption(snopt, "Major Feasibility Tolerance", 5e-6)
    prog.SetSolverOption(snopt, "Major Optimality Tolerance", 1e-4)
    prog.SetSolverOption(snopt, "Superbasics limit", 2000)
    prog.SetSolverOption(snopt, "Linesearch tolerance", 0.9)
    # prog.SetSolverOption(snopt, 'Print file', 'snopt.out')

    result = Solve(prog)
    print(result.get_solver_id().name())
    print(result.is_success())  # We expect this to be false if iterations are limited.

    def HalfStrideToFullStride(a):
        b = PositionView(np.copy(a))

        b.body_y = -a.body_y
        # Mirror quaternion so that roll=-roll, yaw=-yaw
        b.body_qx = -a.body_qx
        b.body_qz = -a.body_qz

        b.front_left_hip_x = -a.front_right_hip_x
        b.front_right_hip_x = -a.front_left_hip_x
        b.rear_left_hip_x = -a.rear_right_hip_x
        b.rear_right_hip_x = -a.rear_left_hip_x

        b.front_left_hip_y = a.front_right_hip_y
        b.front_right_hip_y = a.front_left_hip_y
        b.rear_left_hip_y = a.rear_right_hip_y
        b.rear_right_hip_y = a.rear_left_hip_y

        b.front_left_knee = a.front_right_knee
        b.front_right_knee = a.front_left_knee
        b.rear_left_knee = a.rear_right_knee
        b.rear_right_knee = a.rear_left_knee

        return b

    # Animate trajectory
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    t_sol = np.cumsum(np.concatenate(([0], result.GetSolution(h))))
    q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
    visualizer.StartRecording()
    num_strides = 4
    t0 = t_sol[0]
    tf = t_sol[-1]
    T = tf * num_strides * (2.0 if is_laterally_symmetric else 1.0)
    for t in np.hstack((np.arange(t0, T, 1.0 / 32.0), T)):
        context.SetTime(t)
        stride = (t - t0) // (tf - t0)
        ts = (t - t0) % (tf - t0)
        qt = PositionView(q_sol.value(ts))
        if is_laterally_symmetric:
            if stride % 2 == 1:
                qt = HalfStrideToFullStride(qt)
                qt.body_x += stride_length / 2.0
            stride = stride // 2
        qt.body_x += stride * stride_length
        plant.SetPositions(plant_context, qt[:])
        diagram.ForcedPublish(context)

    visualizer.StopRecording()
    visualizer.PublishRecording()


gait_optimization("swizzle")


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=545d651b-e7e0-4cf4-bfbe-97d6a3f869f8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>